<!DOCTYPE html>
<html lang="fr">
<head>
    <meta charset="UTF-8">
    <title>Analyse de Trajectoires de Tracking</title>
    <style>
        body {
            font-family: Arial, sans-serif;
        }
        h1 {
            color: skyblue;
            font-size: 24px;
        }
        p, li {
            font-size: 16px;
        }
        .green-text{
            color: DarkSeaGreen;
        }
    </style>
</head>
<body>
    <h1>Analyse de Trajectoires de Tracking</h1>
    <p>Ce code d'analyse permet de traiter les données issues du tracking pour extraire et étudier statistiquement les trajectoires pertinentes. Le processus est structuré comme suit :</p>
    <ol>
        <li><strong class="green-text">Récupération des trajectoires :</strong> Collecte de toutes les trajectoires issues des différentes manipulations.</li>
        <li><strong class="green-text">Pré-analyse :</strong> Examen initial des trajectoires pour déterminer celles à conserver pour l'analyse approfondie.</li>
        <li><strong class="green-text">Analyse statistique :</strong> Application de méthodes statistiques aux trajectoires conservées. Les données peuvent être séparées en deux populations pour un traitement spécifique si nécessaire.</li>
    </ol>
    <p>L'analyse se concentre sur les paramètres suivants :</p>
    <ul>
        <li>Angle entre les directions successives d'une particule entre chaque intervalle de temps.</li>
        <li>Vitesses moyennes et instantanées des particules.</li>
        <li>Variation de la vitesse en fonction du temps d'incubation.</li>
        <li>Effet potentiel de la densité de cellules sur la motilité.</li>
    </ul>
</body>
</html>


##
<center><span style="color: seagreen; font-size: 50px; font-style: bold">Chargement et préparation des données.</span></center>
<span style="color: skyblue; font-size: 20px; font-style: bold">Chargement des librairies.</span>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Mar  1 12:46:56 2023.

@author: souchaud
"""

import os
import time
import math
import glob
import cv2
import warnings
import importlib
import pandas as pd
import numpy as np
import trackpy as tp
import seaborn as sns
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import functions_analyze as lib
from typing import List, Optional, Union, Any, Dict, Tuple
from matplotlib import colormaps
from cycler import cycler
from colorama import init
from scipy.stats import linregress
from scipy.optimize import curve_fit

# Reload custom library
importlib.reload(lib)

# Initialize colorama
init(autoreset=True)

# Suppress specific warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=FutureWarning)

# Set default matplotlib style
plt.style.use('default')

<span style="color: skyblue; font-size: 20px; font-style: bold">Paramètres de graphs.</span>

In [ ]:
# plt.rcParams.update({
#     # Figure
#     "figure.figsize": (10, 6),  # Taille par défaut de la figure (largeur, hauteur en pouces)
#     "figure.dpi": 100,  # Résolution en points par pouce
#     "figure.facecolor": (0, 0, 0, 1),  # Fond de la figure : noir pur
#     "figure.edgecolor": "white",  # Bordure de la figure en blanc
#     "figure.titlesize": 20,  # Taille de la police pour le titre principal
#     "figure.titleweight": "bold",  # Style de la police pour le titre principal : gras

#     # Axes
#     "axes.facecolor": (0, 0, 0, 1),  # Fond des axes : noir pur
#     "axes.edgecolor": "white",  # Bordure des axes en blanc
#     "axes.linewidth": 2,  # Épaisseur des bordures des axes
#     "axes.titlesize": 16,  # Taille de la police des titres des axes
#     "axes.titleweight": "bold",  # Style de la police pour les titres des axes : gras
#     "axes.labelsize": 14,  # Taille de la police des étiquettes des axes
#     "axes.labelweight": "medium",  # Style de la police des étiquettes : intermédiaire
#     "axes.labelcolor": "white",  # Couleur des étiquettes des axes
#     "axes.prop_cycle": cycler(color=["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]),  # Cycle des couleurs pour les lignes
#     "axes.grid": True,  # Activer la grille
#     "axes.grid.axis": "both",  # Grille pour les deux axes (x et y)
#     "axes.grid.which": "major",  # Grille pour les ticks principaux
#     "grid.color": "gray",  # Couleur des lignes de la grille
#     "grid.linewidth": 0.5,  # Épaisseur des lignes de la grille
#     "grid.alpha": 0.6,  # Transparence des lignes de la grille

#     # Ticks (Graduations)
#     "xtick.color": "white",  # Couleur des ticks sur l'axe x
#     "ytick.color": "white",  # Couleur des ticks sur l'axe y
#     "xtick.labelsize": 16,  # Taille de la police des ticks sur l'axe x
#     "ytick.labelsize": 16,  # Taille de la police des ticks sur l'axe y
#     "xtick.direction": "in",  # Ticks pointant vers l'intérieur sur l'axe x
#     "ytick.direction": "in",  # Ticks pointant vers l'intérieur sur l'axe y
#     "xtick.major.size": 8,  # Longueur des ticks principaux sur l'axe x
#     "ytick.major.size": 8,  # Longueur des ticks principaux sur l'axe y
#     "xtick.minor.size": 4,  # Longueur des ticks secondaires sur l'axe x
#     "ytick.minor.size": 4,  # Longueur des ticks secondaires sur l'axe y
#     "xtick.major.width": 1.5,  # Épaisseur des ticks principaux sur l'axe x
#     "ytick.major.width": 1.5,  # Épaisseur des ticks principaux sur l'axe y

#     # Lignes et marqueurs
#     # "lines.linewidth": 2,  # Épaisseur par défaut des lignes
#     # "lines.linestyle": "-",  # Style par défaut des lignes continues
#     # "lines.color": "#1f77b4",  # Couleur par défaut des lignes
#     # "lines.marker": "o",  # Marqueur par défaut : cercle
#     # "lines.markersize": 8,  # Taille par défaut des marqueurs
#     # "lines.markeredgewidth": 1.5,  # Épaisseur du bord des marqueurs
#     # "lines.markerfacecolor": "blue",  # Couleur du remplissage des marqueurs

#     # Polices
#     "font.size": 12,  # Taille globale de la police
#     "font.family": "sans-serif",  # Famille de polices par défaut
#     "font.sans-serif": ["Arial", "Helvetica", "DejaVu Sans"],  # Liste des polices sans-serif préférées
#     "text.color": "white",  # Couleur du texte

#     # Légendes
#     "legend.loc": "upper right",  # Emplacement par défaut de la légende
#     "legend.fontsize": 12,  # Taille de la police pour la légende
#     "legend.frameon": True,  # Activer le cadre autour de la légende
#     "legend.framealpha": 0.8,  # Transparence du cadre de la légende
#     "legend.edgecolor": "white",  # Couleur de la bordure de la légende
#     "legend.facecolor": (0.2, 0.2, 0.2, 0.9),  # Fond de la légende : gris foncé semi-transparent

#     # Sauvegarde des graphiques
#     "savefig.dpi": 300,  # Résolution par défaut pour les fichiers sauvegardés
#     "savefig.format": "png",  # Format par défaut pour les fichiers sauvegardés
#     "savefig.facecolor": (0, 0, 0, 1),  # Fond des figures sauvegardées : noir pur
#     "savefig.edgecolor": "none",  # Pas de bordure pour les figures sauvegardées
#     "savefig.transparent": True,  # Fond transparent pour les fichiers sauvegardés

#     # Couleurs et cycles
#     "image.cmap": "viridis",  # Palette par défaut pour les images
# })

plt.rcParams.update({
    # Figure
    "figure.figsize": (7, 5),  # Taille classique pour une figure d'article
    "figure.dpi": 300,
    "figure.facecolor": "white",
    "figure.edgecolor": "white",
    "figure.titlesize": 14,
    "figure.titleweight": "bold",

    # Axes
    "axes.facecolor": "white",
    "axes.edgecolor": "black",
    "axes.linewidth": 1,
    "axes.titlesize": 12,
    "axes.titleweight": "bold",
    "axes.labelsize": 11,
    "axes.labelweight": "normal",
    "axes.labelcolor": "black",
    "axes.prop_cycle": cycler(color=[
        "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b"
    ]),
    "axes.grid": False,

    # Ticks
    "xtick.color": "black",
    "ytick.color": "black",
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "xtick.direction": "out",
    "ytick.direction": "out",
    "xtick.major.size": 5,
    "ytick.major.size": 5,
    "xtick.major.width": 1,
    "ytick.major.width": 1,

    # Lignes
    "lines.linewidth": 1.5,
    "lines.markersize": 6,

    # Police
    "font.size": 11,
    "font.family": "sans-serif",
    "font.sans-serif": ["Arial", "Helvetica", "DejaVu Sans"],
    "text.color": "black",

    # Légendes
    "legend.loc": "best",
    "legend.fontsize": 10,
    "legend.frameon": False,

    # Sauvegarde
    "savefig.dpi": 600,
    "savefig.format": "png",
    "savefig.facecolor": "white",
    "savefig.edgecolor": "white",
    "savefig.transparent": False,

    # Images
    "image.cmap": "viridis",
})

##
<span style="color: skyblue; font-size: 20px; font-style: bold">Initialisation des variables et constantes de travail..</span>

In [ ]:
# Set initial time
INITIAL_TIME = time.time()

# Experiment parameters
TIME_FRAME = 15
SIZE_PIX = 0.637
FPS = 1 / TIME_FRAME

# File to study
file_name = 'filtered_final'
N_FRAME_MIN_STUDY = 200

# Study parameters
ROLLING_MEAN = False
PIXELISATION = False
TIME_FRAME_STUDY = False
DRIFT = False

# Plot parameters
IMG_TYPE = 'png'
ALPHA = 0.5
LINEWIDTH = 0.1
COLOR_SUP = 'blue'
COLOR_INF = 'red'
color_sup_inf = (COLOR_SUP, COLOR_INF)

##
<span style="color: skyblue; font-size: 20px; font-style: bold">Définition des path et dossiers de travails / enregistrements.</span>

<span style="color: skyblue; font-size: 20px; font-style: bold">Conditions des manips étudiées.</span>

<span style="color: skyblue; font-size: 20px; font-style: bold">Défintions et créations des différents dossiers d'enregistrements.</span>

In [ ]:
# General paths
GENERAL_PATH = '/Users/souchaud/Desktop/Analyses/'
GENERAL_PATH_PICTURES = '/Users/souchaud/Desktop/A_analyser/'

# Condition
CONDITION_simple = 'CytoOne_HL5_10x'
CONDITION = f'{CONDITION_simple}_results_tracking'

# Get list of experiments
PATHWAY_EXPERIMENT = [f for f in os.listdir(GENERAL_PATH + CONDITION)
                      if os.path.isdir(os.path.join(GENERAL_PATH + CONDITION, f))]

# Update experiment paths
PATHWAY_EXPERIMENT = [os.path.join(GENERAL_PATH, CONDITION, elem, 'mosaic')
                      for elem in PATHWAY_EXPERIMENT]

# Path to save pictures
path_save_pic = os.path.join(GENERAL_PATH, f'résultats_{CONDITION}_All')

# Create directory if it doesn't exist
os.makedirs(path_save_pic, exist_ok=True)
os.chdir(path_save_pic)


<span style="color: skyblue; font-size: 20px; font-style: bold">Ajout des temps d'incubation.</span>

In [ ]:
# Add incubation times to DATA
conditions_to_values = {
    'ASMOT127': 4.25, 'ASMOT128': 23.58, 'ASMOT130': 29.58, 'ASMOT132': 4.33, 'ASMOT133': 23.12,
    'ASMOT134': 26.12, 'ASMOT135': 29, 'ASMOT136': 31.12, 'ASMOT137': 47.3, 'ASMOT138': 49.75,
    'ASMOT139': 52.25, 'ASMOT140': 0, 'ASMOT141': 4.08, 'ASMOT142': 16.25, 'ASMOT143': 18.67,
    'ASMOT144': 21.08, 'ASMOT145': 23.83, 'ASMOT146': 42.17, 'ASMOT147': 51.17, 'ASMOT148': 47.17,
    'ASMOT149': 70.92, 'ASMOT150': 66.67, 'ASMOT151': 71.17, 'ASMOT152': 76.33, 'ASMOT153': 78.33,
    'ASMOT154': 94.75, 'ASMOT155': 98.42, 'ASMOT156': 100.25, 'ASMOT157': 46.25, 'ASMOT158': 48.08,
    'ASMOT159': 69.58, 'ASMOT160': 20.25, 'ASMOT161': 22.42, 'ASMOT162': 92.33, 'ASMOT163': 0.0,
    'ASMOT164': 5.25, 'ASMOT165': 23.08, 'ASMOT166': 20.08, 'ASMOT167': 21.08, 'ASMOT168': 21.08,
    'ASMOT169': 21.08, 'ASMOT170': 4.58, 'ASMOT171': 7.00, 'ASMOT172': 24.00, 'ASMOT173': 25.58,
    'ASMOT174': 30.58, 'ASMOT175': 47.58, 'ASMOT176': 71.83, 'ASMOT177': 76.67, 'ASMOT178': 77.67,
    'ASMOT179': 95.08, 'ASMOT180': 97.58, 'ASMOT181': 21.08,
}

experiment_to_dell= {
    'ASMOT163',
    'ASMOT161',
    'ASMOT164',
}


##
<center><span style="color: Crimson; font-size: 30px; font-style: bold">Lecture des données expériementales.</span></center>

<span style="color: skyblue; font-size: 20px">On décide de travailler que sur un certain nombre de frame. Ici je décide de travailler sur les 340 première frames pour normaliser les expériences. 
Donc la cellules doit être suivi sur N_MIN_STUDY sur les 340 premières frames. </span>

<span style="color: skyblue; font-size: 20px">Application de fonctions pour moyenne flissante et pixelisation et étude d'une frame sur x </span>

In [ ]:
# Read HDF5 data
importlib.reload(lib)
DATA = lib.read_hdf5_all(
    pathway_experiment=PATHWAY_EXPERIMENT,
    name_file=file_name,
    nbr_frame_min=N_FRAME_MIN_STUDY,
    condition=CONDITION,
    drift=DRIFT,
    search_range=20,
    memory=5
)
# Vérifier si le dictionnaire `experiment_to_dell` n'est pas vide
if experiment_to_dell:
    print(f"Suppression des expériences : {experiment_to_dell}")
    
    # Supprimer les expériences spécifiées
    DATA = DATA[~DATA['experiment'].isin(experiment_to_dell)]
    DATA.reset_index(drop=True, inplace=True)
    print(f"Nombre d'expériences restantes après suppression des expériences ratées: {DATA['experiment'].nunique()}")
else:
    print("Aucune expérience à supprimer, `experiment_to_dell` est vide.")

# Sort DATA by 'frame'
DATA.sort_values(by='frame', inplace=True)
DATA

In [ ]:
print(f"Longueur de DATA après tri sur la vitesse : {len(DATA)}")


In [ ]:
SIZE_PIX = 0.637      # µm/pixel
TIME_FRAME = 15       # secondes par frame
SPEED_THRESHOLD_UM_PER_MIN = 15
SPEED_THRESHOLD_UM_PER_S = SPEED_THRESHOLD_UM_PER_MIN / 60  # ≈ 0.333 µm/s

def compute_speed_filter(df):
    # Tri par particule et frame
    df = df.sort_values(['particle', 'frame']).copy()
    dx = df.groupby('particle')['x'].diff()
    dy = df.groupby('particle')['y'].diff()
    dt = df.groupby('particle')['frame'].diff() * TIME_FRAME  # en secondes
    dist_um = np.sqrt((dx * SIZE_PIX) ** 2 + (dy * SIZE_PIX) ** 2)
    speed_um_s = dist_um / dt
    speed_um_s = speed_um_s.fillna(0)
    df['speed_um_s'] = speed_um_s
    df['is_suspect'] = df['speed_um_s'] > SPEED_THRESHOLD_UM_PER_S
    df_clean = df[~df['is_suspect']].copy()
    return df_clean, df

def speed_filter_iterative(df, speed_thresh_um_s, max_iter=10):
    for i in range(max_iter):
        df_clean, df_with_speed = compute_speed_filter(df)
        n_removed = len(df) - len(df_clean)
        print(f"Iteration {i+1}: removed {n_removed} points")
        if n_removed == 0:
            break
        df = df_clean
    return df

print(f"Longueur de DATA avant tri sur la vitesse : {len(DATA)}")
print(f"avec {DATA['particle'].nunique()} particules")
# Utilisation :
DATA = speed_filter_iterative(DATA, SPEED_THRESHOLD_UM_PER_S)
print(f"Longueur de DATA après tri sur la vitesse : {len(DATA)}")
print(f"avec {DATA['particle'].nunique()} particules")

In [ ]:
# def split_trajectories(df, gap_thresh=1):
#     df = df.sort_values(['particle', 'frame']).copy()
#     # Prépare une colonne pour les nouveaux IDs
#     new_particles = []
#     new_id = 0

#     for pid, group in df.groupby('particle'):
#         frames = group['frame'].to_numpy()
#         # Calcule les gaps entre frames
#         diffs = np.diff(frames, prepend=frames[0])
#         current_id = new_id
#         for i, gap in enumerate(diffs):
#             if i == 0 or gap > gap_thresh:
#                 current_id = new_id
#                 new_id += 1
#             new_particles.append(current_id)
#     df['particle'] = new_particles
#     return df

# # 1. Découpe les trajectoires à chaque trou (trackpy attribue un nouvel ID à chaque fragment consécutif)
# DATA = split_trajectories(DATA, gap_thresh=1)

# # 2. (optionnel mais très conseillé) : Ne garde que les fragments d'au moins X frames
# traj_lengths = DATA.groupby('particle')['frame'].count()
# min_frames = 5  # par exemple
# particles_to_keep = traj_lengths[traj_lengths >= min_frames].index
# DATA = DATA[DATA['particle'].isin(particles_to_keep)].copy()

In [ ]:
# Filter DATA
print("Nombre de particules avant tri: ", DATA['particle'].nunique())
DATA = DATA[DATA['frame'] < 340]

# Keep particles with sufficient frames
DATA = DATA.groupby('particle').filter(lambda x: len(x) >= N_FRAME_MIN_STUDY)
print("Nombre de particules après tri: ", DATA['particle'].nunique())

# Apply optional data transformations
if ROLLING_MEAN:
    DATA = lib.rolling_mean(datas=DATA, roll=3)
if PIXELISATION:
    DATA = lib.pixelisation(datas=DATA, size_pix=SIZE_PIX)
if TIME_FRAME_STUDY:
    DATA, TIME_FRAME = lib.keep_nth_image(traj=DATA, n=N_FRAME_MIN_STUDY, time_frame=TIME_FRAME)

# Calculate instant velocities
DATA['time (min)'] = DATA['frame'] * TIME_FRAME / 60
DATA = lib.vit_instant_new(traj=DATA, lag_time=TIME_FRAME, pix_size=SIZE_PIX, triage=1)

DATA['time to incubation (hours)'] = DATA['experiment'].map(conditions_to_values).fillna(0.0)

In [ ]:
# Supposons que DATA soit ton DataFrame filtré
# On compte le nombre de frames pour chaque particule
traj_lengths = DATA.groupby('particle')['frame'].count()

# Histogramme
plt.figure(figsize=(8, 5))
plt.hist(traj_lengths, bins=50, alpha=0.7)
plt.xlabel('Nombre de frames suivies')
plt.ylabel('Nombre de cellules (trajectoires)')
plt.title('Durée de suivi des cellules (après filtrage)')
plt.show()

# Optionnel : statistiques
print("Statistiques des durées de suivi (en frames) :")
print(traj_lengths.describe())

In [ ]:
# 1. Calcul du nombre de frames par particle
traj_lengths = DATA.groupby('particle')['frame'].count()

# 2. Garder uniquement les particles avec >= 200 frames
particles_to_keep = traj_lengths[traj_lengths >= 200].index

# 3. Filtrer le DataFrame
DATA_filtered = DATA[DATA['particle'].isin(particles_to_keep)].copy()

print(f"Nombre de trajectories gardées : {len(particles_to_keep)}")
print(f"Nombre total de points après filtrage : {len(DATA_filtered)}")

In [ ]:
DATA.columns

In [ ]:
DATA =DATA_filtered
colonnes_a_garder = [
    'y', 'x', 'mass', 'size', 'ecc', 'signal', 'raw_mass', 'ep',
    'frame', 'old_particle', 'count_x', 'experiment', 'condition',
    'count_y', 'particle', 'position', 'time (min)',
    'time to incubation (hours)'
]

DATA = DATA[colonnes_a_garder]
DATA = lib.vit_instant_new(traj=DATA, lag_time=TIME_FRAME, pix_size=SIZE_PIX, triage=1)

In [ ]:
# On filtre juste la colonne 'VitInst [um/min]' :
# (Mettre à NaN les vitesses > 20)
DATA['VitInst [um/min]'] = DATA['VitInst [um/min]'].where(
    (DATA['VitInst [um/min]'] < 20) | (DATA['VitInst [um/min]'].isna()), np.nan
)

# Exemples de stats, NaN seront automatiquement ignorés
mean_speed = DATA['VitInst [um/min]'].mean()
median_speed = DATA['VitInst [um/min]'].median()
print(f"Mean speed: {mean_speed:.2f} µm/min, Median speed: {median_speed:.2f} µm/min")

plt.figure(figsize=(8,5))
plt.hist(DATA['VitInst [um/min]'].dropna(), bins=500, alpha=0.7, density=True)
plt.title("Distribution des vitesses instantanées (<20 µm/min)")
plt.xlabel("VitInst [um/min]")
plt.ylabel("Density")
plt.show()

In [ ]:
DATA.columns
test = DATA['displacement [pix]']*0.637
test = test / DATA['time (min)']

plt.figure(figsize=(8,5))
plt.hist(test, bins=10, alpha=0.7, density=True)
plt.title("Distribution des vitesses instantanées (<20 µm/min)")
plt.xlabel("VitInst [um/min]")
plt.ylabel("Density")
plt.show()

In [ ]:
DATA['experiment'].nunique()

In [ ]:
# # === PARAMÈTRES ===
# base_image_dir = "/Users/souchaud/Desktop/A_analyser/CytoOne_HL5_10x_faits/"
# output_dir = "/Users/souchaud/Desktop/faibles/"
# os.makedirs(output_dir, exist_ok=True)

# # 1. Calcul de la première position de chaque particule
# first_positions = DATA.sort_values('frame').groupby('particle', as_index=False).first()

# # 2. Génération des images par expérience avec annotation des premières positions
# for experiment in first_positions['experiment'].unique():
#     sub_df = first_positions[first_positions['experiment'] == experiment].copy()

#     # Localise le dossier de l’expérience
#     experiment_dirs = glob.glob(os.path.join(base_image_dir, f"*{experiment}*"))
#     if not experiment_dirs:
#         print(f"Dossier pour {experiment} non trouvé !")
#         continue
#     experiment_dir = experiment_dirs[0]

#     # Charge l'image frame 0
#     img_path = os.path.join(experiment_dir, "mosaic/mosaic_total_0.tif")
#     if not os.path.isfile(img_path):
#         print(f"Image {img_path} non trouvée !")
#         continue

#     img = cv2.imread(img_path)
#     if img is None:
#         print(f"Image {img_path} illisible !")
#         continue

#     # Trace les cercles sur chaque première position
#     for _, row in sub_df.iterrows():
#         x = int(round(row["x"]))
#         y = int(round(row["y"]))
#         cv2.circle(img, (x, y), 50, (0, 0, 255), 3)

#     # Sauvegarde
#     out_path = os.path.join(output_dir, f"{experiment}_mosaic_total_0_first_positions.png")
#     cv2.imwrite(out_path, img)
#     print(f"Image annotée sauvegardée : {out_path}")

In [ ]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# === 0. Clustering sur mass, size, ecc ===
X = DATA[['mass', 'size', 'ecc']].dropna()
X_scaled = StandardScaler().fit_transform(X)
kmeans = KMeans(n_clusters=2, random_state=0).fit(X_scaled)
DATA['cluster'] = -1
DATA.loc[X.index, 'cluster'] = kmeans.labels_

# === Affichage du clustering (facultatif) ===
plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=DATA, x='size', y='mass',
    hue='cluster', palette=['green', 'orange'],
    alpha=0.1, s=0.001, marker='+', linewidth=1.0
)
plt.title("K-means clustering (2 groupes)")
plt.grid()
plt.show()


Préparation et entourer les particules avec un low MSD.

In [ ]:
# # === PARAMÈTRES ===
# base_image_dir = "/Users/souchaud/Desktop/A_analyser/CytoOne_HL5_10x_faits/"
# output_dir = "/Users/souchaud/Desktop/faibles/"
# min_distance = 10  # en pixels pour filtrage spatial
# os.makedirs(output_dir, exist_ok=True)

# # === 1. Extraire particules du cluster 1 uniquement ===
# data_cluster1 = DATA[DATA['cluster'] == 1].copy()

# # === 2. Trouver la première frame de chaque particule ===
# first_frame_df = data_cluster1.sort_values('frame').groupby('particle', as_index=False).first()
# first_frame_df['start_frame'] = first_frame_df['frame']  # juste pour clarté

# # === 3. Filtrage spatial dans chaque frame de début ===
# filtered_particles = []

# for f in first_frame_df['start_frame'].unique():
#     same_frame = first_frame_df[first_frame_df['start_frame'] == f].copy()
    
#     if len(same_frame) <= 1:
#         filtered_particles.append(same_frame)
#         continue

#     coords = same_frame[['x', 'y']].to_numpy()
#     nbrs = NearestNeighbors(n_neighbors=2).fit(coords)
#     distances, _ = nbrs.kneighbors(coords)
#     keep_mask = distances[:, 1] > min_distance
#     filtered_particles.append(same_frame[keep_mask])

# # === 4. Particules uniques à annoter ===
# final_df = pd.concat(filtered_particles, ignore_index=True)

# # === 5. Génération des images par expérience ===
# for experiment in final_df['experiment'].unique():
#     sub_df = final_df[final_df['experiment'] == experiment].copy()

#     # Localise le dossier de l’expérience
#     experiment_dirs = glob.glob(os.path.join(base_image_dir, f"*{experiment}*"))
#     if not experiment_dirs:
#         print(f"Dossier pour {experiment} non trouvé !")
#         continue
#     experiment_dir = experiment_dirs[0]

#     # Charge l'image frame 0
#     img_path = os.path.join(experiment_dir, "mosaic/mosaic_total_0.tif")
#     if not os.path.isfile(img_path):
#         print(f"Image {img_path} non trouvée !")
#         continue

#     img = cv2.imread(img_path)
#     if img is None:
#         print(f"Image {img_path} illisible !")
#         continue

#     # Trace les cercles
#     for _, row in sub_df.iterrows():
#         x = int(round(row["x"]))
#         y = int(round(row["y"]))
#         cv2.circle(img, (x, y), 50, (0, 0, 255), 3)

#     # Sauvegarde
#     out_path = os.path.join(output_dir, f"{experiment}_mosaic_total_0_cluster1_filtered.png")
#     cv2.imwrite(out_path, img)
#     print(f"Image annotée sauvegardée : {out_path}")

###
<center><span style="color: skyblue; font-size: 20px"> Mean Mass and size plot </span></center>

In [ ]:
# Plot Mean Mass and Size per Manipulation
def plot_mean_mass_size(DATA, path_save_pic, IMG_TYPE):
    manips = DATA['experiment'].unique()
    num_manips = len(manips)
    fig = plt.figure(figsize=(16, 2 * num_manips))
    gs = gridspec.GridSpec(num_manips, 3, fig)
    colors = ['skyblue', 'lightgreen', 'salmon']
    for i, manip in enumerate(manips):
        data_manip = DATA[DATA['experiment'] == manip]
        mass_means = data_manip.groupby('particle')['mass'].mean()
        size_means = data_manip.groupby('particle')['size'].mean()
        filtered_data = data_manip[data_manip['frame'] == 0]

        ax1 = fig.add_subplot(gs[i, 0])
        ax1.hist(mass_means, bins=100, color=colors[0], density=True)
        ax1.set_title(f"Mean mass of particles for {manip}")
        ax1.set_xlabel("Mean mass")
        ax1.set_ylabel("Density")

        ax2 = fig.add_subplot(gs[i, 1])
        ax2.hist(size_means, bins=100, color=colors[1], density=True)
        ax2.set_title(f"Mean size of particles for {manip}")
        ax2.set_xlabel("Mean size")
        ax2.set_ylabel("Density")

        ax3 = fig.add_subplot(gs[i, 2])
        ax3.scatter(filtered_data['size'], filtered_data['mass'], c="#d62728", edgecolors="#d62728", alpha=0.7)
        ax3.set_title(f"Mass vs. Size at frame 0 for {manip}")
        ax3.set_xlabel("Size")
        ax3.set_ylabel("Mass")
        ax3.grid(True, linestyle="--", alpha=0.5)

    plt.tight_layout()
    plt.show()
    fig.savefig(os.path.join(path_save_pic, f"Mean_Mass_Size_manip.{IMG_TYPE}"), format=IMG_TYPE)

plot_mean_mass_size(DATA, path_save_pic, IMG_TYPE='svg')

###
<center><span style="color: skyblue; font-size: 20px"> plot total path for each partciel in each experiment (histograms) </span></center>

In [ ]:
# Calculate total path for the first N frames
path_data = lib.calculate_total_path_first_frames(DATA, first_n_frames=100)

# Plot Total Path in First 100 Frames per Experiment
def plot_total_path(path_data):
    grouped = path_data.groupby('experiment')
    n_experiments = len(grouped)
    fig, axes = plt.subplots(nrows=n_experiments, figsize=(10, 2 * n_experiments))
    axes = axes if n_experiments > 1 else [axes]
    for (experiment, group), ax in zip(grouped, axes):
        ax.hist(group['total_path_first_n'], bins=50, range=[0, 100], density=True, alpha=0.5)
        ax.set_title(f"Total path in first 100 frames for {experiment}")
        ax.set_xlabel('Length path (μm)')
        ax.set_ylabel('Density')
    plt.tight_layout()
    plt.savefig(os.path.join(path_save_pic, f"Total path in first 100 frames.{IMG_TYPE}"), format=IMG_TYPE)
    plt.show()

plot_total_path(path_data)

###
<center><span style="color: skyblue; font-size: 20px"> Creation des traj centrées / distance cumulée / IMSD </span></center>


In [ ]:
# Center trajectories
DATA.reset_index(inplace=True)
DATA = lib.center(traj=DATA)

print(f"\n\nTemps de préparation des données pour {CONDITION}: {time.time() - INITIAL_TIME} sec\n\n")

# Calculate total and cumulative displacement
DATA, start_end = lib.length_displacement(traj=DATA, size_pix=SIZE_PIX)

# Compute MSD and cutoff
DATA2 = DATA.copy()
DATA2['frame'] = pd.factorize(DATA2['frame'])[0]
IMSD = tp.imsd(traj=DATA2, mpp=SIZE_PIX, fps=FPS, max_lagtime=200, statistic='msd')

###
<center><span style="color: skyblue; font-size: 20px"> Premier fit pour exclure certaines traj </span></center>

In [ ]:
# Trajectory clustering with fit and defining a cutoff
LAG_TIME_FIT = 5
importlib.reload(lib)
COEF_INF, COEF_SUP, PART_COEF_INF, PART_COEF_SUP, CUTOFF = lib.traj_clustering_with_fit_cutoff(
    DATA2, imsd=IMSD, hist=True, lag_time_fit=LAG_TIME_FIT, micronperpixel=SIZE_PIX,
    fps=FPS, binsize=250, peak_height=50, peak_width=1, save=True, pathway_fig=path_save_pic,
    name='all_experiment_autocorr', img_type=IMG_TYPE, plot=True, color_sup_inf=color_sup_inf,
    cutoff_default=0
)

# Keep only particles above cutoff
DATA = DATA[DATA['particle'].isin(PART_COEF_SUP)]

In [ ]:
DATA = DATA[DATA['VitInst [um/min]'] < 20]
DATA

###
<center><span style="color: skyblue; font-size: 20px"> Nombre de particules par frame pour les expériences. </span></center>

ça n'a pas un grand interêt ici, mais c'est par principe pour vérification et compréhension d'éventuels phenomènes.

In [ ]:
# Plot Number of Particles per Frame for All Experiments in One Graph
def plot_nbr_particles_per_frame_combined(DATA, path_save_pic, IMG_TYPE):
    """
    Plot the number of unique particles per frame for all experiments on a single graph with different colors.

    Parameters:
    - DATA (DataFrame): The input data containing tracking information.
    - path_save_pic (str): The path where the plot will be saved.
    - IMG_TYPE (str): The format for saving the plot (e.g., 'png', 'jpg').
    """
    experiments = DATA['experiment'].unique()

    # Generate a colormap with enough unique colors for all experiments
    colormap = plt.colormaps['tab20']  # Use the modern colormap API
    colors = [colormap(i / len(experiments)) for i in range(len(experiments))]

    plt.figure(figsize=(12, 13))
    for i, exp in enumerate(experiments):
        # Group data by time and calculate the number of unique particles per frame
        nbr_part_per_frame = DATA[DATA['experiment'] == exp].groupby('time (min)')['particle'].nunique()
        plt.plot(
            nbr_part_per_frame.index, nbr_part_per_frame.values, 
            label=exp, color=colors[i], alpha=1, linewidth=0.8
        )

    # Add labels, legend, and title
    plt.title('Number of Particles per Frame for All Experiments', fontsize=14)
    plt.xlabel('Time (min)', fontsize=12)
    plt.ylabel('Number of Particles', fontsize=12)
    plt.legend(title="Experiments", bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()

    # Save and show the plot
    fig_path = os.path.join(path_save_pic, f"Nbr_particle_per_Frame_combined.{IMG_TYPE}")
    plt.savefig(fig_path, format=IMG_TYPE, bbox_inches='tight')
    plt.show()

    print(f"Plot saved to {fig_path}")

# Appel de la fonction
plot_nbr_particles_per_frame_combined(DATA, path_save_pic, IMG_TYPE)

In [ ]:
# # Plot Number of Particles per Frame per Experiment
# def plot_nbr_particles_per_frame(DATA, path_save_pic, IMG_TYPE):
#     experiments = DATA['experiment'].unique()
#     n_cols = 2
#     n_rows = (len(experiments) + n_cols - 1) // n_cols
#     fig, axs = plt.subplots(n_rows, n_cols, figsize=(20 * n_cols, 10 * n_rows))
#     axs = axs.flatten()
#     for i, exp in enumerate(experiments):
#         nbr_part_per_frame = DATA[DATA['experiment'] == exp].groupby('time (min)')['particle'].nunique()
#         ax = axs[i]
#         ax.plot(nbr_part_per_frame.index, nbr_part_per_frame.values)
#         ax.set_ylim([0, nbr_part_per_frame.max() + 10])
#         ax.set_title(f'Nbr particle per Frame - {exp}')
#         ax.set_xlabel('Time (min)')
#         ax.set_ylabel('Number of particles')
#     for ax in axs[len(experiments):]:
#         ax.axis('off')
#     plt.tight_layout()
#     plt.savefig(os.path.join(path_save_pic, f"Nbr_particle_per_Frame_manip_par_manip.{IMG_TYPE}"), format=IMG_TYPE)
#     plt.show()

# plot_nbr_particles_per_frame(DATA, path_save_pic, IMG_TYPE)

###
<center><span style="color: skyblue; font-size: 20px"> Nombre de particules par frame pour toutes les paticules. </span></center>


In [ ]:
# Plot Number of Particles per Frame
nbr_part_per_frame = DATA.groupby('time (min)')['particle'].nunique()
lib.plot_datas(
    x_values=nbr_part_per_frame.index,
    y_values=nbr_part_per_frame.values,
    title='Nbr particles per Frame',
    x_label='Time (min)', y_label='Number of particles',
    x_lim=[0, nbr_part_per_frame.index.max()], y_lim=[0, 10000],
    save=True, path_save_pic=path_save_pic, img_type=IMG_TYPE
)

###
<center><span style="color: skyblue; font-size: 40px">Trajectoires toutes rassemblées. </span></center>


In [ ]:
# Plot Trajectories after Removing Suspicious Particles
fig, axis = plt.subplots(figsize=(12, 12), dpi=300)
axis.set_aspect('equal', 'box')
plt.title('Trajectories after removing suspicious particles', fontsize=16)
tp.plot_traj(DATA, label=False, ax=axis)
for line in axis.get_lines():
    line.set_linewidth(0.1)
plt.show()
fig.savefig(os.path.join(path_save_pic, f'Trajectories_after_removing_suspicious_particles.{IMG_TYPE}'),
            format=IMG_TYPE, dpi=300, bbox_inches='tight')

In [ ]:
# Plot centered Trajectories after Removing Suspicious Particles
fig, axis = plt.subplots(figsize=(12, 12), dpi=300)
axis.set_aspect('equal', 'box')
plt.title('Centered trajectories after removing suspicious particles', fontsize=16)
tp.plot_traj(DATA[['Xc [pix]', 'Yc [pix]', 'frame', 'particle']].rename(columns={'Xc [pix]': 'x', 'Yc [pix]': 'y'}), label=False, ax=axis)
for line in axis.get_lines():
    line.set_linewidth(0.2)
plt.show()
fig.savefig(os.path.join(path_save_pic, f'Centered trajectories_after_removing_suspicious_particles.{IMG_TYPE}'),
            format=IMG_TYPE, dpi=300, bbox_inches='tight')

In [ ]:
# Plot Trajectories on Original Frames
def plot_trajectories_on_frames(DATA, path_save_pic, GENERAL_PATH_PICTURES, CONDITION_simple):
    image_path = os.path.join(GENERAL_PATH_PICTURES, f"{CONDITION_simple}_faits")
    plot_exp = DATA.groupby('experiment')
    num_experiments = len(plot_exp)
    num_cols = 2
    num_rows = (num_experiments + num_cols - 1) // num_cols
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, num_rows * 5))
    axes = axes.flatten()
    for ax, (exp_name, exp_data) in zip(axes, plot_exp):
        exp_directories = []
        for dirpath, dirnames, _ in os.walk(image_path):
            for dirname in dirnames:
                if exp_name in dirname:
                    full_path = os.path.join(dirpath, dirname)
                    exp_directories.append(full_path)
        if exp_directories:
            image_path_directory = os.path.join(exp_directories[0], 'mosaic', 'mosaic_total_0.tif')
            frame = imageio.imread(image_path_directory)
            ax.set_aspect('equal', 'box')
            ax.set_title(f'Trajectories for {exp_name}')
            tp.plot_traj(exp_data, superimpose=frame, label=False, ax=ax)
        else:
            print(f"No directory found for {exp_name}")
            ax.axis('off')
    plt.tight_layout()
    plt.show()
    fig.savefig(os.path.join(path_save_pic, 'trajectories_on_frame_all_experiment.pdf'), format='pdf')

plot_trajectories_on_frames(DATA, path_save_pic, GENERAL_PATH_PICTURES, CONDITION_simple)

###
<center><span style="color: skyblue; font-size: 20px"> Vitesse instantanée moyenne par frame. </span></center>


In [ ]:
# Plot Mean Instantaneous Speed per Frame
mean_VitInst_per_frame = DATA.groupby('time (min)')['VitInst [um/min]'].median()
mean_VitInst_per_frame = mean_VitInst_per_frame.rolling(5).mean().dropna()

# Appel avec vérification du répertoire
lib.plot_datas(
    x_values=mean_VitInst_per_frame.index,
    y_values=mean_VitInst_per_frame.values,
    title='Mean VitInst [um/min] per Frame',
    x_label='Time (min)', y_label='Mean VitInst [um/min]',
    x_lim=[0, mean_VitInst_per_frame.index.max()], y_lim=[0, 10],
    save=True, path_save_pic=path_save_pic, img_type=IMG_TYPE
)

In [ ]:
# Recompute MSD
# DATA_intermediaire = DATA.copy()
DATA['frame'] = pd.factorize(DATA['frame'])[0]
IMSD = tp.imsd(traj=DATA, mpp=SIZE_PIX, fps=FPS, max_lagtime=200, statistic='msd')

lib.plot_msd(IMSD, fps=FPS, name="MSD of all frames vs lag time (s)",
             color_plot='forestgreen', save=True, pathway_saving=path_save_pic,
             alpha=0.3, linewidth=0.1, img_type=IMG_TYPE)

In [ ]:
# Reload custom library
importlib.reload(lib)

In [ ]:
# Recompute trajectory clustering with new cutoff
LAG_TIME_FIT = 5
COEF_INF, COEF_SUP, PART_COEF_INF, PART_COEF_SUP, CUTOFF = lib.traj_clustering_with_fit_cutoff(
    DATA, imsd=IMSD, hist=True, lag_time_fit_start=0, lag_time_fit=LAG_TIME_FIT , micronperpixel=SIZE_PIX,
    fps=FPS, binsize=250, peak_height=50, peak_width=1, save=True, pathway_fig=path_save_pic,
    name='MSD and slopes', img_type=IMG_TYPE, plot=True, color_sup_inf=color_sup_inf,
    cutoff_default=0.6
)

# Ajouter les colonnes 'is_inf' et 'is_sup' à DATA
DATA['particle'] = DATA['particle'].astype(int)
PART_COEF_INF = set(map(int, PART_COEF_INF))
PART_COEF_SUP = set(map(int, PART_COEF_SUP))

# Ajout des colonnes 'is_inf' et 'is_sup'
DATA['is_inf'] = DATA['particle'].isin(PART_COEF_INF).astype(int)
DATA['is_sup'] = DATA['particle'].isin(PART_COEF_SUP).astype(int)


In [ ]:
import matplotlib.pyplot as plt

# Pour éviter les effets de labels/marqueurs automatiques
def plot_trajectories(data, color, ax, linewidth=0.2):
    for pid, group in data.groupby('particle'):
        ax.plot(group['Xc [pix]'], group['Yc [pix]'], color=color, lw=linewidth)

fig, axis = plt.subplots(figsize=(12, 12), dpi=300)
axis.set_aspect('equal', 'box')
plt.title('Centered trajectories after removing suspicious particles', fontsize=16)

# 2. Trajectoires bleues (PART_COEF_SUP)
data_sup = DATA[DATA['particle'].isin(PART_COEF_SUP)]
plot_trajectories(data_sup, color='blue', ax=axis, linewidth=0.2)

# 1. Trajectoires rouges (PART_COEF_INF)
data_inf = DATA[DATA['particle'].isin(PART_COEF_INF)]
plot_trajectories(data_inf, color='red', ax=axis, linewidth=0.2)



plt.xlabel('Xc [pix]')
plt.ylabel('Yc [pix]')
plt.show()

# Enregistrement
fig.savefig(
    os.path.join(path_save_pic, f'Centered_trajectories_grouped.{IMG_TYPE}'),
    format=IMG_TYPE, dpi=300, bbox_inches='tight'
)

In [ ]:
data_coef_sup = DATA[DATA['particle'].isin(PART_COEF_INF)]
data_coef_sup['experiment'].unique()

In [ ]:
# Plot centered Trajectories after Removing Suspicious Particles
fig, axis = plt.subplots(figsize=(12, 12), dpi=300)
axis.set_aspect('equal', 'box')
plt.title('Centered trajectories after removing suspicious particles', fontsize=16)
data_inf = DATA[DATA['particle'].isin(PART_COEF_INF)]
tp.plot_traj(data_inf[['Xc [pix]', 'Yc [pix]', 'frame', 'particle']].rename(columns={'Xc [pix]': 'x', 'Yc [pix]': 'y'}), label=False, ax=axis)
for line in axis.get_lines():
    line.set_linewidth(0.2)
plt.show()
# fig.savefig(os.path.join(path_save_pic, f'Centered trajectories_after_removing_suspicious_particles.{IMG_TYPE}'),
#             format=IMG_TYPE, dpi=300, bbox_inches='tight')

In [ ]:
print("Nb de particules total :", DATA['particle'].nunique())
print("Nb particules INF demandées :", len(PART_COEF_INF))
print("Nb réellement trouvées :", DATA[DATA['particle'].isin(PART_COEF_INF)]['particle'].nunique())
# ─────────────────────────────────────────────────────────────────────────────
#  MSD MOYENNE : GLOBAL vs PART_COEF_INF vs PART_COEF_SUP
# ─────────────────────────────────────────────────────────────────────────────
MAX_LAG = 200   # le même que précédemment (en frames)

# --- 2) MSD moyenne des particules INF ------------------
DATA_inf  = DATA[DATA['particle'].isin(PART_COEF_INF)]
# --- 3) MSD moyenne des particules SUP ------------------
DATA_sup  = DATA[DATA['particle'].isin(PART_COEF_SUP)]

EMSD_inf = tp.emsd(traj=DATA[DATA['particle'].isin(PART_COEF_INF)], mpp=SIZE_PIX, fps=FPS, max_lagtime=MAX_LAG, detail=True)
EMSD_sup = tp.emsd(traj=DATA[DATA['particle'].isin(PART_COEF_SUP)], mpp=SIZE_PIX, fps=FPS, max_lagtime=MAX_LAG, detail=True)
EMSD = tp.emsd(traj=DATA, mpp=SIZE_PIX, fps=FPS, max_lagtime=MAX_LAG, detail=True)

In [ ]:
# Vérifie que les DataFrames ont bien les colonnes attendues
assert 'msd' in EMSD.columns, "La colonne 'msd' est absente de EMSD"
assert 'msd' in EMSD_inf.columns, "La colonne 'msd' est absente de EMSD_inf"
assert 'msd' in EMSD_sup.columns, "La colonne 'msd' est absente de EMSD_sup"

# Tracé des MSD moyennes
plt.figure(figsize=(7, 5))

plt.plot(EMSD.index, EMSD['msd'], label='Global', color='black', linewidth=2)
plt.plot(EMSD_inf.index, EMSD_inf['msd'], label='PART_COEF_INF', color='red', linewidth=1.5)
plt.plot(EMSD_sup.index, EMSD_sup['msd'], label='PART_COEF_SUP', color='blue', linewidth=1.5)

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Lag time (s)')
plt.ylabel('MSD (µm²)')
plt.title('Mean MSD - Global vs INF vs SUP (via tp.emsd)')
plt.legend()
plt.grid(True, which='both', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Nombre de points à utiliser pour les fits
N_points = 10

# Préparer le plot
plt.figure(figsize=(10, 7))

# Courbes à tracer
courbes = {
    # 'Global': EMSD,
    'PART_COEF_INF': EMSD_inf,
    'PART_COEF_SUP': EMSD_sup
}

# Couleurs associées
couleurs = {
    # 'Global': 'black',
    'PART_COEF_INF': 'red',
    'PART_COEF_SUP': 'blue'
}

# Tracer les courbes MSD + les fits
for label, df in courbes.items():
    lag = np.array(df['lagt'])
    msd = np.array(df['msd'])

    # Tracer la courbe principale
    plt.plot(lag, msd, label=label, color=couleurs[label], linewidth=1)

    # === Fit court temps ===
    log_lag_start = np.log10(lag[:3])
    log_msd_start = np.log10(msd[:3])
    slope_start, intercept_start, _, _, _ = linregress(log_lag_start, log_msd_start)
    fit_start = 10 ** (slope_start * np.log10(lag[:]) + intercept_start)

    plt.plot(lag[:], fit_start, '--', color=couleurs[label],
             linewidth=1.5, label=f"{label} fit début\nα={slope_start:.2f} and intercept={intercept_start:.2f}")

    # === Fit moyen temps ===
    log_lag_end = np.log10(lag[5:12])
    log_msd_end = np.log10(msd[5:12])
    slope_end, intercept_end, _, _, _ = linregress(log_lag_end, log_msd_end)
    fit_end = 10 ** (slope_end * np.log10(lag[:]) + intercept_end)

    plt.plot(lag[:], fit_end, ':', color=couleurs[label],
             linewidth=2, label=f"{label} fit fin\nα={slope_end:.2f} and intercept={intercept_end:.2f}")

# Mise en forme du graphe
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Lag time (s)', fontsize=14)
plt.ylabel('MSD (µm²)', fontsize=14)
plt.title('MSD + Régressions début et fin (log-log)', fontsize=16)
plt.grid(True, which='both', ls='--', alpha=0.4)
plt.legend(fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import linregress
import numpy as np
import matplotlib.pyplot as plt

N_FIRST = 5  # fit points 0 à 4
N_NEXT = 5   # fit points 5 à 9
N_LAST = 100  # fit points -10 à la fin

# --- FIT sur les 5 premiers points ---
log_lag_first = np.log10(lag[:N_FIRST])
log_msd_first = np.log10(msd[:N_FIRST])
slope_first, intercept_first, *_ = linregress(log_lag_first, log_msd_first)
fit_first = 10 ** (slope_first * np.log10(lag) + intercept_first)

# --- FIT sur les points 6 à 10 (indices 5 à 9) ---
log_lag_next = np.log10(lag[N_FIRST:N_FIRST+N_NEXT])
log_msd_next = np.log10(msd[N_FIRST:N_FIRST+N_NEXT])
slope_next, intercept_next, *_ = linregress(log_lag_next, log_msd_next)
fit_next = 10 ** (slope_next * np.log10(lag) + intercept_next)

# --- FIT sur les 10 derniers points ---
log_lag_last = np.log10(lag[-N_LAST:])
log_msd_last = np.log10(msd[-N_LAST:])
slope_last, intercept_last, *_ = linregress(log_lag_last, log_msd_last)
fit_last = 10 ** (slope_last * np.log10(lag) + intercept_last)

# --- Tracé ---
plt.figure(figsize=(9, 6))
plt.plot(lag, msd, 'ko-', label='MSD brute')
plt.plot(lag, fit_first, 'b--', label=f'Fit points 1-5\nα={slope_first:.2f}')
plt.plot(lag, fit_next, 'r--', label=f'Fit points 6-10\nα={slope_next:.2f}')
plt.plot(lag, fit_last, 'g--', label=f'Fit 10 derniers points\nα={slope_last:.2f}')

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Lag time (s)')
plt.ylabel('MSD (µm²)')
plt.title('MSD brute + fits log-log courts')
plt.grid(True, which='both', ls='--')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
#  FONCTION 1 : AUTOCORRÉLATION D’UNE SÉRIE 1-D
# ─────────────────────────────────────────────────────────────────────────────
def autocorr_msd(msd: np.ndarray, max_lag: int = None) -> pd.Series:
    """
    Calcule l'autocorrélation normalisée d'une série MSD (1-D).

    Parameters
    ----------
    msd : np.ndarray
        Valeurs MSD(t) pour un seul lag (1-D).
    max_lag : int, optional
        Nombre maximal de lags à considérer (en frames). Par défaut : N-1.

    Returns
    -------
    pd.Series
        Autocorrélation normalisée C(τ) avec τ∈[0, max_lag] (index = lag).
    """
    msd = np.asarray(msd, dtype=float)
    N = len(msd)
    if max_lag is None or max_lag >= N:
        max_lag = N - 1

    mu  = msd.mean()
    var = msd.var()
    if var == 0:
        return pd.Series([1.0] + [0.0]*max_lag, index=range(max_lag + 1))

    acov = np.array([
        np.dot(msd[:N-lag] - mu, msd[lag:] - mu) / (N - lag)
        for lag in range(max_lag + 1)
    ])
    acorr = acov / var
    return pd.Series(acorr, index=range(max_lag + 1), name="autocorr_msd")

# ─────────────────────────────────────────────────────────────────────────────
#  FONCTION 2 : AUTOCORRÉLATION À PARTIR DE trackpy.imsd
# ─────────────────────────────────────────────────────────────────────────────
def msd_autocorrelation_from_imsd(imsd: pd.DataFrame, fps: float,
                                  max_lag: int = None,
                                  agg: str = 'mean') -> pd.Series:
    """
    Calcule l'autocorrélation normalisée de la MSD agrégée (mean/median)
    à partir du DataFrame renvoyé par trackpy.imsd.

    Parameters
    ----------
    imsd : pd.DataFrame
        Sortie de trackpy.imsd (index = lag, colonnes = particules).
    fps : float
        Taux d'acquisition (images par seconde).
    max_lag : int, optional
        Nombre maximal de lags (en frames) pour l'autocorrélation.
    agg : {'mean', 'median'}
        Méthode d’agrégation des MSD sur les particules.

    Returns
    -------
    pd.Series
        Autocorrélation normalisée indexée par le lag en secondes.
    """
    if agg == 'mean':
        msd_series = imsd.mean(axis=1).values
    elif agg == 'median':
        msd_series = imsd.median(axis=1).values
    else:
        raise ValueError("agg must be 'mean' or 'median'")

    ac = autocorr_msd(msd_series, max_lag=max_lag)

    # Conversion index (frames) → secondes
    ac.index = ac.index / fps
    ac.index.name = "lag_seconds"
    ac.name = f"autocorr_msd_{agg}"
    return ac

# ─────────────────────────────────────────────────────────────────────────────
#  EXEMPLE D’UTILISATION
# ─────────────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    # EXEMPLE : suppose que 'DATA' est votre DataFrame de trajectoires
    # et que vous avez déjà défini SIZE_PIX et FPS.
    #
    # 1) Calcul MSD (trackpy)
    IMSD = tp.imsd(traj=DATA,
                   mpp=SIZE_PIX,   # micron par pixel
                   fps=FPS,
                   max_lagtime=200,
                   statistic='msd')   # => DataFrame (lags × particules)

    # 2) Autocorrélation normalisée (mean sur particules)
    ac_msd = msd_autocorrelation_from_imsd(IMSD, fps=FPS,
                                           max_lag=100, agg='mean')

    # 3) Tracé
    plt.figure(figsize=(8, 4))
    plt.plot(ac_msd.index, ac_msd.values, marker='o')
    plt.xlabel("Lag (s)")
    plt.ylabel("Autocorrélation normalisée de la MSD")
    plt.title("Autocorrélation de la MSD (moyenne sur particules)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

###
<center><span style="color: skyblue; font-size: 40px">Calculs des différents metrics importants </span></center>


In [ ]:
def calculate_metrics(DATA):
    # Calculer les vitesses moyennes instantanées pour toutes les particules
    mean_vitinst_df = DATA.groupby(['experiment', 'particle'])['VitInst [um/min]'].mean().reset_index()
    mean_vitinst_df = mean_vitinst_df.rename(columns={'VitInst [um/min]': 'mean_vit_inst'})

    # Calculer les médianes des vitesses instantanées pour toutes les particules
    median_vitinst_df = DATA.groupby(['experiment', 'particle'])['VitInst [um/min]'].median().reset_index()
    median_vitinst_df = median_vitinst_df.rename(columns={'VitInst [um/min]': 'median_vit_inst'})

    # Initialisation d'une liste pour stocker les résultats
    metrics = []

    # Parcours de chaque particule pour calculer les métriques
    for particle, group in DATA.groupby('particle'):
        experiment = group['experiment'].iloc[0]

        # Limiter aux 200 premières lignes
        limited_group = group.head(200)

        # Somme des déplacements (seulement sur les 200 premières lignes)
        displacement_sum = limited_group['displacement [pix]'].sum()

        # Récupérer la vitesse moyenne instantanée pré-calculée
        mean_vit_inst = mean_vitinst_df.loc[
            (mean_vitinst_df['experiment'] == experiment) & (mean_vitinst_df['particle'] == particle),
            'mean_vit_inst'
        ].values[0]

        # Récupérer la médiane des vitesses instantanées pré-calculée
        median_vit_inst = median_vitinst_df.loc[
            (median_vitinst_df['experiment'] == experiment) & (median_vitinst_df['particle'] == particle),
            'median_vit_inst'
        ].values[0]

        # Distance entre la position de départ et d'arrivée
        start_position = limited_group.iloc[0][['x', 'y']].values
        end_position = limited_group.iloc[-1][['x', 'y']].values
        start_end_distance = np.linalg.norm(end_position - start_position)

        # Stocker les résultats dans un dictionnaire
        metrics.append({
            'experiment': experiment,
            'particle': particle,
            'displacement_sum [um]': displacement_sum,
            'mean_vit_inst [um/min]': mean_vit_inst,
            'median_vit_inst [um/min]': median_vit_inst,
            'start_end_distance [um]': start_end_distance
        })

    # Conversion en DataFrame
    metrics_df = pd.DataFrame(metrics)
    return metrics_df

# Appel de la fonction pour obtenir le DataFrame consolidé
metrics_df = calculate_metrics(DATA)

# Affichage pour vérification
metrics_df.head(10)

In [ ]:
def calculate_all_metrics(DATA):
    """
    Calcule diverses métriques pour chaque expérience, y compris la vitesse moyenne
    des particules sup et inf.
    """
    # Calculer le temps d'incubation et le nombre de particules (exp_hours)
    exp_hours = (
        DATA.groupby('experiment')
        .agg({
            'time to incubation (hours)': 'first',
            'particle': 'nunique'
        })
        .reset_index()
        .rename(columns={'particle': 'number_of_particles'})
    )

    # Compter les particules is_inf et is_sup
    particle_counts = (
        DATA.groupby(['experiment', 'particle'])
        .agg(
            is_inf=('is_inf', 'max'),
            is_sup=('is_sup', 'max')
        )
        .reset_index()
        .groupby('experiment')
        .agg(
            number_of_inf=('is_inf', 'sum'),
            number_of_sup=('is_sup', 'sum')
        )
        .reset_index()
    )
    particle_counts['proportion_inf'] = particle_counts['number_of_inf'] / (
        particle_counts['number_of_inf'] + particle_counts['number_of_sup']
    )

    # Calculer les métriques issues de metrics_df
    metrics_df = DATA.groupby(['experiment', 'particle']).apply(
        lambda group: {
            'displacement_sum [um]': group.head(200)['displacement [pix]'].sum(),
            'start_end_distance [um]': np.linalg.norm(
                group.head(200).iloc[0][['x', 'y']].values -
                group.head(200).iloc[-1][['x', 'y']].values
            ),
            'mean_vit_inst [um/min]': group['VitInst [um/min]'].median()
        }
    ).apply(pd.Series).reset_index()

    # Ajouter la vitesse mediane pour les particules sup et inf
    sup_inf_speeds = DATA.groupby(['experiment', 'particle']).agg(
        is_sup=('is_sup', 'max'),
        is_inf=('is_inf', 'max'),
        mean_vit_inst=('VitInst [um/min]', 'median')
    ).reset_index()

    # Calculer la vitesse mediane des particules sup et inf par expérience
    sup_inf_speeds_agg = sup_inf_speeds.groupby('experiment').agg(
        mean_speed_sup=('mean_vit_inst', lambda x: x[sup_inf_speeds['is_sup'] == 1].median() if (sup_inf_speeds['is_sup'] == 1).any() else 0),
        mean_speed_inf=('mean_vit_inst', lambda x: x[sup_inf_speeds['is_inf'] == 1].median() if (sup_inf_speeds['is_inf'] == 1).any() else 0)
    ).reset_index()

    # Calculer taille et particules par champ
    spatial_metrics = DATA.groupby('experiment').apply(lambda group: {
        'taille': math.ceil(group['y'].max() / 2048) * math.ceil(group['x'].max() / 2048),
        'nombre_part_par_champs': group['particle'].nunique() /
        (math.ceil(group['y'].max() / 2048) * math.ceil(group['x'].max() / 2048)) if (
            math.ceil(group['y'].max() / 2048) * math.ceil(group['x'].max() / 2048)) > 0 else 0
    }).apply(pd.Series).reset_index()

    # Calculer d'autres métriques
    result_metrics = metrics_df.groupby('experiment').agg(
        displacement_sum_mean=('displacement_sum [um]', 'median'),
        start_end_distance_mean=('start_end_distance [um]', 'median'),
        mean_speed=('mean_vit_inst [um/min]', 'median')
    ).reset_index()

    # Fusionner toutes les métriques
    consolidated_metrics = (
        exp_hours
        .merge(particle_counts, on='experiment', how='outer')
        .merge(result_metrics, on='experiment', how='outer')
        .merge(sup_inf_speeds_agg, on='experiment', how='outer')
        .merge(spatial_metrics, on='experiment', how='outer')
    )

    # Renommer les colonnes après la fusion
    consolidated_metrics = consolidated_metrics.rename(columns={
        'displacement_sum_mean': 'displacement_sum_mean [um]',
        'start_end_distance_mean': 'start_end_distance_mean [um]',
        'mean_speed': 'mean_speed [um/min]',
        'mean_speed_sup': 'mean_speed_sup [um/min]',
        'mean_speed_inf': 'mean_speed_inf [um/min]'
    })

    return consolidated_metrics

# Appel de la fonction
all_metrics_df = calculate_all_metrics(DATA)

# Affichage pour vérification
all_metrics_df.head(10)

In [ ]:
print(DATA[DATA['is_inf']==True]['particle'].nunique())
print(DATA['particle'].nunique())

In [ ]:
# Plot Displacement
importlib.reload(lib)
lib.plot_displacement(
    DATA,
    start_end=metrics_df[['particle', 'start_end_distance [um]']],
    alpha=0.1,
    linewidth=0.3,
    ylim=[0, 750],
    xlim=[0, DATA['time (min)'].max()],
    save=True,
    pathway_saving=path_save_pic,
    name='displacement start-end all',
    img_type=IMG_TYPE
)

In [ ]:
# Plot Histograms of Start-End Distances
def plot_histograms(start_end, PART_COEF_SUP, PART_COEF_INF, IMG_TYPE):
    fig, axes = plt.subplots(3, 1, figsize=(5, 10), sharex=True, sharey=True)
    axes[0].hist(metrics_df['start_end_distance [um]'], bins=250, color='royalblue', alpha=0.7, density=True)
    axes[0].set_xlim([0, 400])
    axes[0].set_ylim([0, 0.03])
    axes[0].set_title('Start-End Distances - All Particles', fontsize=16)
    axes[0].set_ylabel('Density', fontsize=14)
    axes[0].grid(True, linestyle='--', alpha=0.6)

    axes[1].hist(metrics_df[metrics_df['particle'].isin(PART_COEF_SUP)][['start_end_distance [um]']], bins=250, color='blue', alpha=0.7, density=True)
    axes[1].set_xlim([0, 400])
    axes[1].set_title('Start-End Distances - High Coefficient Particles', fontsize=16)
    axes[1].set_ylabel('Density', fontsize=14)
    axes[1].grid(True, linestyle='--', alpha=0.6)

    axes[2].hist(metrics_df[metrics_df['particle'].isin(PART_COEF_INF)][['start_end_distance [um]']], bins=250, color='red', alpha=0.7, density=True)
    axes[2].set_xlim([0, 400])
    axes[2].set_title('Start-End Distances - Low Coefficient Particles', fontsize=16)
    axes[2].set_xlabel('Distance (μm)', fontsize=14)
    axes[2].set_ylabel('Density', fontsize=14)
    axes[2].grid(True, linestyle='--', alpha=0.6)

    plt.tight_layout()
    plt.savefig(os.path.join(path_save_pic, f"Nstart_end inf supp and all.{IMG_TYPE}"), format=IMG_TYPE)
    plt.show()

plot_histograms(start_end, PART_COEF_SUP, PART_COEF_INF, IMG_TYPE=IMG_TYPE)

In [ ]:
# Plot Results in Function of Number of Particles per Field
def plot_results_vs_particles(all_metrics_df, path_save_pic, CONDITION_simple, img_type='svg'):
    fig, axes = plt.subplots(1, 2, figsize=(20, 6))
    ax1, ax2 = axes.flatten()
    ax1.scatter(all_metrics_df['nombre_part_par_champs'], all_metrics_df['displacement_sum_mean [um]'], marker='+', color='orange')
    ax1.set_title('Average distance traveled vs. Number of particles per field')
    ax1.set_xlabel('Number of particles per field')
    ax1.set_ylabel('Average distance traveled')
    ax2.scatter(all_metrics_df['nombre_part_par_champs'], all_metrics_df['mean_speed [um/min]'], marker='+', color='blue')
    ax2.set_title('Mean Speed vs. Number of particles per field')
    ax2.set_xlabel('Number of particles per field')
    ax2.set_ylabel('Mean Speed [μm/min]')
    plt.tight_layout()
    plt.show()
    fig.savefig(os.path.join(path_save_pic, f"Results_vs_particles_{CONDITION_simple}.{img_type}"), format=img_type)

plot_results_vs_particles(all_metrics_df, path_save_pic, CONDITION_simple, img_type=IMG_TYPE)

In [ ]:
# Compute Angle Changes between Directions
def compute_angle_changes(DATA, PART_COEF_SUP, PART_COEF_INF):
    def angle_between_directions(row):
        dx1, dy1 = row['dir_x'], row['dir_y']
        dx2, dy2 = row['dir_x_next'], row['dir_y_next']
        angle_initial = np.arctan2(dy1, dx1)
        angle_final = np.arctan2(dy2, dx2)
        angle_change = angle_final - angle_initial
        angle_change = (angle_change + np.pi) % (2 * np.pi) - np.pi
        return np.degrees(angle_change)

    df_sup = DATA[DATA['particle'].isin(PART_COEF_SUP)].copy()
    df_sup.sort_values(by=['particle', 'frame'], inplace=True)
    df_sup['dir_x'] = df_sup.groupby('particle')['x'].diff().fillna(0)
    df_sup['dir_y'] = df_sup.groupby('particle')['y'].diff().fillna(0)
    df_sup['dir_x_next'] = df_sup.groupby('particle')['dir_x'].shift(-1)
    df_sup['dir_y_next'] = df_sup.groupby('particle')['dir_y'].shift(-1)
    df_sup['angle_change'] = df_sup.apply(angle_between_directions, axis=1)
    df_sup.dropna(subset=['dir_x_next', 'dir_y_next'], inplace=True)

    df_inf = DATA[DATA['particle'].isin(PART_COEF_INF)].copy()
    df_inf.sort_values(by=['particle', 'frame'], inplace=True)
    df_inf['dir_x'] = df_inf.groupby('particle')['x'].diff().fillna(0)
    df_inf['dir_y'] = df_inf.groupby('particle')['y'].diff().fillna(0)
    df_inf['dir_x_next'] = df_inf.groupby('particle')['dir_x'].shift(-1)
    df_inf['dir_y_next'] = df_inf.groupby('particle')['dir_y'].shift(-1)
    df_inf['angle_change'] = df_inf.apply(angle_between_directions, axis=1)
    df_inf.dropna(subset=['dir_x_next', 'dir_y_next'], inplace=True)

    return df_sup, df_inf

df_sup, df_inf = compute_angle_changes(DATA, PART_COEF_SUP, PART_COEF_INF)

In [ ]:
# Plot histograms of angle changes
def plot_angle_histograms(df_all, df_sup, df_inf, img_type='svg'):
    fig, axes = plt.subplots(3, 1, figsize=(5, 10), sharex=True)
    axes[0].hist(df_all['angle_change'], bins=1000, alpha=0.5, color='green')
    axes[0].set_title('Angle Changes - All Particles')
    axes[0].set_xlabel('Angle (degrees)')
    axes[0].set_ylabel('Density')

    axes[1].hist(df_sup['angle_change'], bins=1000, color='blue', alpha=0.5)
    axes[1].set_title('Angle Changes - High Coefficient Particles')
    axes[1].set_xlabel('Angle (degrees)')
    axes[1].set_ylabel('Density')

    axes[2].hist(df_inf['angle_change'], bins=1000, color='red', alpha=0.5)
    axes[2].set_title('Angle Changes - Low Coefficient Particles')
    axes[2].set_xlabel('Angle (degrees)')
    axes[2].set_ylabel('Density')

    plt.tight_layout()
    plt.show()
    
    fig.savefig(os.path.join(path_save_pic, f'Angle_Changes_Histograms.{img_type}'), format=img_type)

df_all = pd.concat([df_sup, df_inf])
plot_angle_histograms(df_all, df_sup, df_inf, img_type=IMG_TYPE)

In [ ]:
def plot_mean_vit_histograms_comparison(metrics_df, part_coef_sup, part_coef_inf, alpha=0.5, path_save_pic=None, img_type='png'):
    """
    Trace sur le même graphique les histogrammes des mean_vit_inst [um/min]
    pour les particules avec coefficients supérieurs et inférieurs.

    Parameters:
    - metrics_df (DataFrame): DataFrame contenant les données, avec une colonne 'mean_vit_inst [um/min]'.
    - part_coef_sup (list): Liste des particules avec coefficients supérieurs.
    - part_coef_inf (list): Liste des particules avec coefficients inférieurs.
    - path_save_pic (str, optional): Chemin pour sauvegarder l'image. Si None, l'image ne sera pas sauvegardée.
    - img_type (str, optional): Format de l'image pour la sauvegarde (par défaut 'png').

    Returns:
    None
    """
    # Vérification de la présence de la colonne nécessaire
    if 'mean_vit_inst [um/min]' not in metrics_df.columns:
        raise ValueError("La colonne 'mean_vit_inst [um/min]' est absente de metrics_df.")
    
    # Filtrer les données
    df_sup = metrics_df[metrics_df['particle'].isin(part_coef_sup)]
    df_inf = metrics_df[metrics_df['particle'].isin(part_coef_inf)]
    
    median_value_sup = df_sup['median_vit_inst [um/min]'].median()
    median_value_inf = df_inf['median_vit_inst [um/min]'].median()

    # Préparer le graphique
    plt.figure(figsize=(10, 10))
    plt.hist(df_sup['median_vit_inst [um/min]'], bins=100, color='blue', alpha=alpha, label="hight slope", density=True)
    plt.hist(df_inf['median_vit_inst [um/min]'], bins=100, color='red', alpha=alpha, label="low slope", density=True)

     # Ajouter une barre verticale à la médiane
    plt.axvline(median_value_inf, color='red', linestyle='--', linewidth=2, label=f'Median: {median_value_inf:.2f} μm/min')
    plt.axvline(median_value_sup, color='blue', linestyle='--', linewidth=2, label=f'Median: {median_value_sup:.2f} μm/min')


    # Ajouter des titres et des légendes
    plt.title("Median speed [μm/min]", fontsize=14)
    plt.xlabel("Median Instantaneous Velocity [μm/min]", fontsize=12)
    plt.ylabel("Density", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(fontsize=12)
    
    # Sauvegarde de l'image si un chemin est spécifié
    if path_save_pic:
        save_path = os.path.join(path_save_pic, f'Median_Velocity_Histograms_Comparison.{img_type}')
        plt.savefig(save_path, format=img_type)
        print(f"Plot sauvegardé à : {save_path}")
    
    # Afficher le graphique
    plt.tight_layout()
    plt.show()

In [ ]:
plot_mean_vit_histograms_comparison(metrics_df, PART_COEF_SUP, PART_COEF_INF, path_save_pic=path_save_pic, img_type=IMG_TYPE, alpha=0.7)

In [ ]:
DATA

In [ ]:
def plot_vit_inst_histograms_comparison(metrics_df, part_coef_sup, part_coef_inf, alpha=0.5, path_save_pic=None, img_type='png'):
    """
    Trace sur le même graphique les histogrammes des mean_vit_inst [um/min]
    pour les particules avec coefficients supérieurs et inférieurs.

    Parameters:
    - metrics_df (DataFrame): DataFrame contenant les données, avec une colonne 'mean_vit_inst [um/min]'.
    - part_coef_sup (list): Liste des particules avec coefficients supérieurs.
    - part_coef_inf (list): Liste des particules avec coefficients inférieurs.
    - path_save_pic (str, optional): Chemin pour sauvegarder l'image. Si None, l'image ne sera pas sauvegardée.
    - img_type (str, optional): Format de l'image pour la sauvegarde (par défaut 'png').

    Returns:
    None
    """
    # Vérification de la présence de la colonne nécessaire
    if 'mean_vit_inst [um/min]' not in metrics_df.columns:
        raise ValueError("La colonne 'mean_vit_inst [um/min]' est absente de metrics_df.")
    
    # Filtrer les données
    df_sup = DATA[DATA['particle'].isin(part_coef_sup)]
    df_inf = DATA[DATA['particle'].isin(part_coef_inf)]
    
    instant_value_sup = df_sup['VitInst [um/min]']
    instant_value_inf = df_inf['VitInst [um/min]']

    # Préparer le graphique
    plt.figure(figsize=(10, 10))
    plt.hist(df_sup['VitInst [um/min]'], bins=100, color='blue', alpha=alpha, label="hight slope", density=True)
    plt.hist(df_inf['VitInst [um/min]'], bins=100, color='red', alpha=alpha, label="low slope", density=True)

    #  # Ajouter une barre verticale à la médiane
    # plt.axvline(median_value_inf, color='red', linestyle='--', linewidth=2, label=f'Median: {median_value_inf:.2f} μm/min')
    # plt.axvline(median_value_sup, color='blue', linestyle='--', linewidth=2, label=f'Median: {median_value_sup:.2f} μm/min')


    # Ajouter des titres et des légendes
    plt.title("Median speed [μm/min]", fontsize=14)
    plt.xlabel("Median Instantaneous Velocity [μm/min]", fontsize=12)
    plt.ylabel("Density", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(fontsize=12)
    
    # Sauvegarde de l'image si un chemin est spécifié
    if path_save_pic:
        save_path = os.path.join(path_save_pic, f'Inst_Velocity_Histograms_Comparison.{img_type}')
        plt.savefig(save_path, format=img_type)
        print(f"Plot sauvegardé à : {save_path}")
    
    # Afficher le graphique
    plt.tight_layout()
    plt.show()

In [ ]:
plot_vit_inst_histograms_comparison(metrics_df, PART_COEF_SUP, PART_COEF_INF, path_save_pic=path_save_pic, img_type=IMG_TYPE, alpha=0.7)

In [ ]:
# Préparer le graphique
plt.figure(figsize=(10, 10))
plt.hist(DATA['VitInst [um/min]'], bins=500, color='limegreen', alpha=0.8, label="all speed", density=True)

# Ajouter une barre verticale à la médiane
# plt.axvline(median_value, color='red', linestyle='--', linewidth=2, label=f'Median: {median_value:.2f} μm/min')

# Ajouter des titres et des légendes
plt.title("VitInst[μm/min]", fontsize=14)
plt.xlabel("Instantaneous Velocity [μm/min]", fontsize=12)
plt.ylabel("Density", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=12)

In [ ]:
def plot_mean_vit_histograms_comparison(metrics_df, path_save_pic=None, img_type='png', alpha=0.7):
    """
    Trace sur le même graphique les histogrammes des mean_vit_inst [um/min]
    pour les particules avec coefficients supérieurs et inférieurs.

    Parameters:
    - metrics_df (DataFrame): DataFrame contenant les données, avec une colonne 'mean_vit_inst [um/min]'.
    - path_save_pic (str, optional): Chemin pour sauvegarder l'image. Si None, l'image ne sera pas sauvegardée.
    - img_type (str, optional): Format de l'image pour la sauvegarde (par défaut 'png').

    Returns:
    None
    """
    # Vérification de la présence de la colonne nécessaire
    if 'median_vit_inst [um/min]' not in metrics_df.columns:
        raise ValueError("La colonne 'median_vit_inst [um/min]' est absente de metrics_df.")
    
    # Calcul de la médiane
    median_value = metrics_df['median_vit_inst [um/min]'].median()
    print(f"Median value: {median_value}")

    # Préparer le graphique
    plt.figure(figsize=(10, 10))
    plt.hist(metrics_df['median_vit_inst [um/min]'], bins=100, color='limegreen', alpha=alpha, label="all speed", density=True)

    # Ajouter une barre verticale à la médiane
    plt.axvline(median_value, color='red', linestyle='--', linewidth=2, label=f'Median: {median_value:.2f} μm/min')

    # Ajouter des titres et des légendes
    plt.title("Median speed [μm/min]", fontsize=14)
    plt.xlabel("Median Instantaneous Velocity [μm/min]", fontsize=12)
    plt.ylabel("Density", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(fontsize=12)
    
    # Sauvegarde de l'image si un chemin est spécifié
    if path_save_pic:
        save_path = os.path.join(path_save_pic, f'Median_Velocity_Histograms_all.{img_type}')
        plt.savefig(save_path, format=img_type)
        print(f"Plot sauvegardé à : {save_path}")
    
    # Afficher le graphique
    # Sans tight_layout (les titres peuvent se chevaucher)
    plt.tight_layout()  # Ajuste automatiquement l'espacement
    plt.show()

In [ ]:
plot_mean_vit_histograms_comparison(metrics_df, path_save_pic=path_save_pic, img_type=IMG_TYPE, alpha=0.5)

In [ ]:
def plot_velocity_histograms(all_metrics_df, metrics_df, path_save_pic, img_type):
    """
    Plot histograms of median instantaneous velocities for each experiment, sorted by incubation time.

    Parameters:
    - all_metrics_df (DataFrame): Aggregated metrics per experiment.
    - metrics_df (DataFrame): Metrics for each particle and experiment.
    - path_save_pic (str): Path where the plot will be saved.
    - img_type (str): Image format for saving the plot.
    """
    # Trier les expériences par temps d'incubation
    exp_hours = all_metrics_df[['experiment', 'time to incubation (hours)']].sort_values(by='time to incubation (hours)')
    experiments = exp_hours['experiment'].tolist()
    incubation_times = exp_hours['time to incubation (hours)'].tolist()

    # Déterminer les bornes de l'axe X
    global_min = metrics_df['median_vit_inst [um/min]'].min()
    global_max = metrics_df['median_vit_inst [um/min]'].max()
    delta = (global_max - global_min) * 0.1  # Ajouter 10% de marge pour l'affichage
    global_min -= delta
    global_max += delta

    # Liste pour stocker les médianes des vitesses moyennes instantanées
    median_values_list = []

    # Nombre d'expériences à afficher
    n_experiments = len(experiments)
    fig, axes = plt.subplots(nrows=n_experiments, ncols=1, figsize=(10, 2 * n_experiments), sharex=True, sharey=True)

    if n_experiments == 1:
        axes = [axes]  # S'assurer que 'axes' est une liste si un seul subplot

    # Palette de couleurs
    palette = colormaps['tab20']
    colors = [palette(i / n_experiments) for i in range(n_experiments)]

    # Boucle sur chaque expérience
    for idx, ax in enumerate(axes):
        exp = experiments[idx]
        hour = incubation_times[idx]

        # Filtrer les données pour cette expérience
        data_exp = metrics_df[metrics_df['experiment'] == exp]
        num_particles = len(data_exp)  # Nombre de particules dans cette expérience

        # Calculer la médiane des vitesses moyennes instantanées pour cette expérience
        median_value = data_exp['median_vit_inst [um/min]'].median()
        median_values_list.append(median_value)  # Stocker la médiane

        # Tracer l'histogramme
        ax.hist(data_exp['median_vit_inst [um/min]'], bins=30, alpha=0.3, range=(global_min, global_max), color=colors[idx])

        # Ajouter titre et labels
        ax.set_title(f'Vit. Inst. Mediane [μm/min] - {exp} (Incubation: {hour}h)')
        ax.set_xlabel('VitInst [μm/min]')
        ax.set_ylabel('Nombre de particules')

        # Ajouter la médiane avec une ligne verticale rouge
        ax.axvline(median_value, color='red', linestyle='dashed', linewidth=1)
        ax.text(
            median_value + 0.05 * (global_max - global_min), 
            # ax.get_ylim()[1] * 0.95,
            75,
            f'Médiane: {median_value:.2f}',
            color='red',
            ha='left'
        )

        # Ajouter le nombre total de particules sur le graphique
        ax.text(
            global_max - 0.1 * (global_max - global_min),  # Position X
            # ax.get_ylim()[1] * 0.85,  # Position Y (haut du graphe)
            75,
            f'Nb Particules: {num_particles}',
            color='black',
            ha='right',
            fontsize=10,
            bbox=dict(facecolor='white', alpha=0.5, edgecolor='black')
        )

        # Ajuster les limites de l'axe X
        ax.set_xlim(global_min, global_max)

    # Calculer la moyenne et la médiane des médianes enregistrées
    overall_median_of_medians = np.median(median_values_list)
    overall_mean_of_medians = np.mean(median_values_list)

    # Afficher les résultats finaux
    print(f"Médiane globale des médianes : {overall_median_of_medians:.2f} μm/min")
    print(f"Moyenne globale des médianes : {overall_mean_of_medians:.2f} μm/min")

    # Ajuster l'affichage et sauvegarder la figure
    plt.tight_layout()
    fig_path = os.path.join(path_save_pic, f'Velocity_Histograms.{img_type}')
    fig.savefig(fig_path, format=img_type)
    plt.show()
    
    print(f"Plot saved to {fig_path}")

    return overall_median_of_medians, overall_mean_of_medians  # Retourner ces valeurs si besoin

In [ ]:
# plot_velocity_histograms(all_mean_vitinst, medians, exp_hours, path_save_pic, IMG_TYPE)
plot_velocity_histograms(all_metrics_df, metrics_df, path_save_pic, IMG_TYPE)

###
<center><span style="color: skyblue; font-size: 20px"> Calcul de l'autocorrelation de la vitesse </span></center>


In [ ]:
def velocity_autocorrelation_per_particle(data, max_lag, fps):
    """
    Calcule l'autocorrélation de la vitesse pour chaque particule individuellement.

    Paramètres:
    ----------
    - data : pd.DataFrame
        DataFrame contenant les trajectoires, avec les colonnes :
        ['experiment', 'particle', 'frame', 'dx [pix]', 'dy [pix]']
    - max_lag : int
        Nombre maximum d'intervalle de temps (tau) pour l'autocorrélation.
    - fps : float
        Nombre d'images par seconde, pour convertir le temps en secondes.

    Retourne:
    ----------
    - autocorr_df : pd.DataFrame
        DataFrame contenant l'autocorrélation moyenne et individuelle par particule.
    """

    # Vérifier que les colonnes nécessaires existent
    required_columns = {'experiment', 'particle', 'frame', 'dx [pix]', 'dy [pix]'}
    if not required_columns.issubset(data.columns):
        raise KeyError(f"🚨 Erreur : Les colonnes requises {required_columns} ne sont pas toutes présentes dans DATA.")

    # Trier les données
    data = data.sort_values(by=['experiment', 'particle', 'frame'])

    # Calculer les composantes de vitesse vx et vy
    data['vx'] = data['dx [pix]'] / data.groupby(['experiment', 'particle'])['frame'].diff()
    data['vy'] = data['dy [pix]'] / data.groupby(['experiment', 'particle'])['frame'].diff()

    # Remplir les NaN initiaux (première frame de chaque particule)
    data[['vx', 'vy']] = data[['vx', 'vy']].fillna(0)

    # Dictionnaire pour stocker les résultats
    autocorr_dict = {'lag': []}
    particle_corrs = {}

    # Parcourir chaque particule et calculer son autocorrélation individuelle
    for particle, group in data.groupby(['experiment', 'particle']):
        group = group.sort_values(by='frame')

        # Vérifier qu'il y a assez de points pour calculer l'autocorrélation
        if len(group) < max_lag:
            continue

        v_t = np.sqrt(group['vx']**2 + group['vy']**2).values

        autocorr_values = []
        for tau in range(1, max_lag + 1):
            v_tau = np.roll(v_t, -tau)  # Décaler les vitesses de tau frames
            valid_idx = np.arange(len(v_t)) < (len(v_t) - tau)  # Éviter les valeurs hors limites

            # Calculer l'autocorrélation
            corr = np.mean(v_t[valid_idx] * v_tau[valid_idx]) / np.mean(v_t[valid_idx]**2)
            autocorr_values.append(corr)

        particle_corrs[particle] = autocorr_values

    # Transformer en DataFrame
    autocorr_df = pd.DataFrame.from_dict(particle_corrs, orient='index', columns=[f'lag_{tau}' for tau in range(1, max_lag + 1)])
    autocorr_df.index.name = 'particle'

    # Calculer la moyenne des autocorrélations par lag
    autocorr_df.loc['mean'] = autocorr_df.mean()

    return autocorr_df

In [ ]:
# Paramètres
max_lag = 30
fps = FPS  # images par seconde
lags = np.arange(1, max_lag + 1) / fps  # en secondes

# Filtrer les données selon les listes d’IDs
data_inf = DATA[DATA['particle'].isin(PART_COEF_INF)]
data_sup = DATA[DATA['particle'].isin(PART_COEF_SUP)]

# --- calculs précédents ---
ac_inf = velocity_autocorrelation_per_particle(data_inf, max_lag=max_lag, fps=fps)
ac_sup = velocity_autocorrelation_per_particle(data_sup, max_lag=max_lag, fps=fps)

# Extraction de la moyenne
mean_inf = ac_inf.loc['mean'].values
mean_sup = ac_sup.loc['mean'].values

# Normalisation : on ramène chaque courbe à 1 au premier lag
norm_inf = mean_inf / mean_inf[0]
norm_sup = mean_sup / mean_sup[0]

# Tracé
plt.figure(figsize=(8,5))
plt.plot(lags, norm_inf, marker='o', label='Inférieure (normalisée)')
plt.plot(lags, norm_sup, marker='s', label='Supérieure (normalisée)')
plt.xlabel("Décalage temporel (s)")
plt.ylabel("Autocorrélation de la vitesse (normalisée)")
plt.title("Autocorrélation normalisée par sous-population")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def velocity_autocorrelation_by_experiment_and_global(data, max_lag, fps):
    """
    Calcule l'autocorrélation de la vitesse par particule, puis :
      1) Moyenne par expérience
      2) Moyenne globale (sur toutes les particules de toutes les expériences).
    
    Retourne un DataFrame dont :
      - L'index = décalages en secondes (1/fps, 2/fps, ..., max_lag/fps).
      - Les colonnes = chaque expérience + la colonne 'All' (moyenne globale).
    
    Paramètres
    ----------
    data : pd.DataFrame
        DataFrame contenant les trajectoires, avec colonnes :
        ['experiment', 'particle', 'frame', 'dx [pix]', 'dy [pix]']
    max_lag : int
        Nombre maximum d'intervalle de temps (tau) pour l'autocorrélation.
    fps : float
        Nombre d'images par seconde, pour convertir frames -> secondes.
    
    Retourne
    --------
    autocorr_df : pd.DataFrame
        * index : lags (s)
        * colonnes : expériences + 'All'
    """
    # Vérifier colonnes
    required_cols = {'experiment', 'particle', 'frame', 'dx [pix]', 'dy [pix]'}
    if not required_cols.issubset(data.columns):
        raise KeyError(
            f"🚨 Les colonnes requises {required_cols} ne sont pas toutes présentes dans 'data'."
        )
    
    # Trier la DataFrame
    data = data.sort_values(by=['experiment', 'particle', 'frame'])
    
    # Calcul des composantes de vitesse vx, vy
    data['delta_frame'] = data.groupby(['experiment', 'particle'])['frame'].diff()
    data['vx'] = data['dx [pix]'] / data['delta_frame']
    data['vy'] = data['dy [pix]'] / data['delta_frame']
    data[['vx', 'vy']] = data[['vx', 'vy']].fillna(0)
    
    # Stockage des autocorrélations
    # experiment_autocorrs[exp_name] = [ [autocorr_part1], [autocorr_part2], ... ]
    experiment_autocorrs = {}
    # Liste globale de toutes les particules, toutes expériences confondues
    all_autocorrs = []
    
    # Calcul de l'autocorrélation par particule
    for (exp_name, particle_id), group in data.groupby(['experiment', 'particle']):
        group = group.sort_values(by='frame')
        
        # Norme de la vitesse
        v_t = np.sqrt(group['vx']**2 + group['vy']**2).values
        
        if len(v_t) < max_lag:
            # Pas assez de points
            continue
        
        # Calcul de l'autocorrélation pour tau = 1..max_lag
        autocorr_values = []
        for tau in range(1, max_lag + 1):
            v_tau = np.roll(v_t, -tau)
            valid_idx = np.arange(len(v_t)) < (len(v_t) - tau)
            
            numerator = np.mean(v_t[valid_idx] * v_tau[valid_idx])
            denominator = np.mean(v_t[valid_idx]**2)
            corr = numerator / denominator if denominator != 0 else np.nan
            autocorr_values.append(corr)
        
        # Stocker dans experiment_autocorrs
        if exp_name not in experiment_autocorrs:
            experiment_autocorrs[exp_name] = []
        experiment_autocorrs[exp_name].append(autocorr_values)
        
        # Stocker aussi dans la liste globale
        all_autocorrs.append(autocorr_values)
    
    # Moyenne par expérience
    experiment_mean_corr = {}
    for exp_name, list_autocorrs in experiment_autocorrs.items():
        arr = np.array(list_autocorrs)  # shape: (nb_particules, max_lag)
        mean_by_lag = np.nanmean(arr, axis=0)
        experiment_mean_corr[exp_name] = mean_by_lag
    
    # Moyenne globale (All) sur toutes les particules de TOUTES les expériences
    if len(all_autocorrs) > 0:
        arr_all = np.array(all_autocorrs)  # shape: (nb_total_particules, max_lag)
        global_mean = np.nanmean(arr_all, axis=0)
    else:
        global_mean = np.full(shape=(max_lag,), fill_value=np.nan)
    
    # Construire le DataFrame final
    lags_in_sec = np.arange(1, max_lag + 1) / fps
    autocorr_df = pd.DataFrame(experiment_mean_corr, index=lags_in_sec)
    autocorr_df.index.name = "lag (s)"
    
    # Ajouter la colonne 'All'
    autocorr_df['All'] = global_mean
    
    return autocorr_df

max_lag = 30
fps = FPS  # par exemple

autocorr_df = velocity_autocorrelation_by_experiment_and_global(DATA, max_lag, fps)

# Création de la figure
plt.figure()#figsize=(8, 5))

# Tracé de "All" en bleu avec un style spécifique
plt.plot(autocorr_df.index, autocorr_df["All"], marker='o', linestyle='-', 
         label="All", color="blue", linewidth=2, alpha=1.0)

# Tracé des expériences avec transparence et ligne fine
for col in autocorr_df.columns:
    if col != "All":  # On exclut "All"
        plt.plot(autocorr_df.index, autocorr_df[col], marker='+', linestyle='-', 
                 alpha=0.5, linewidth=0.5, label=col)

# Ajout de légende unique
# plt.legend(title="Expériences et All", loc="upper right")

# Axes et titre
plt.xlabel("Lag (s)")
plt.ylabel("Autocorrélation")
plt.title("Autocorrélation de la vitesse, par expérience et globale")
plt.grid(True)

# Affichage
plt.show()

Fit exponnentiel de l'autocorrelation de la vitesse

<center><span style="color: skyblue; font-size: 20px"> Mean speed vs time to incubation </span></center>


In [ ]:
# def plot_mean_speed_with_sup_inf(all_metrics_df, path_save_pic, xlim: list = None):
#     """
#     Plot mean speed, mean speed sup, and mean speed inf as a function of incubation time
#     on the same graph.

#     Parameters:
#     - all_metrics_df (DataFrame): DataFrame containing aggregated metrics for each experiment.
#                                   Must include columns: 'time to incubation (hours)',
#                                   'mean_speed [um/min]', 'mean_speed_sup [um/min]', 
#                                   and 'mean_speed_inf [um/min]'.
#     - path_save_pic (str): Path where the plot will be saved.
#     - xlim (list, optional): Limit for the x-axis.

#     Returns:
#     None
#     """
#     # Trier les données par temps d'incubation
#     all_metrics_sorted = all_metrics_df.sort_values(by='time to incubation (hours)')

#     # Préparer le tracé
#     plt.figure(figsize=(10, 5))

#     # Courbe pour la vitesse moyenne totale
#     plt.plot(
#         all_metrics_sorted['time to incubation (hours)'],
#         all_metrics_sorted['mean_speed [um/min]'],
#         linestyle='--', color='green', alpha=0.6,
#         label='Mean Speed (Total)'
#     )

#     # Courbe pour les particules sup
#     plt.plot(
#         all_metrics_sorted['time to incubation (hours)'],
#         all_metrics_sorted['mean_speed_sup [um/min]'],
#         linestyle='--', color='blue', alpha=0.6,
#         label='Mean Speed (Sup)'
#     )

#     # Courbe pour les particules inf
#     plt.plot(
#         all_metrics_sorted['time to incubation (hours)'],
#         all_metrics_sorted['mean_speed_inf [um/min]'],
#         linestyle='--', color='red', alpha=0.6,
#         label='Mean Speed (Inf)'
#     )

#     # Ajouter des labels et des titres
#     plt.title('Mean Speed vs. Time to Incubation (Total, Sup, Inf)', fontsize=14)
#     plt.xlabel('Time to Incubation (hours)', fontsize=12)
#     plt.ylabel('Mean Speed [μm/min]', fontsize=12)
#     plt.ylim([0, 15])  # Limites pour l'axe des y
#     if xlim:
#         plt.xlim(xlim)  # Limites pour l'axe des x (optionnel)
#     plt.grid(True, linestyle='--', alpha=0.6)
#     plt.legend(fontsize=10)

#     # Sauvegarder et afficher
#     plt.tight_layout()
#     fig_path = os.path.join(path_save_pic, 'Mean_Speed_vs_Time_Sup_Inf.png')
#     plt.savefig(fig_path, format='png')
#     plt.show()

#     print(f"Plot saved to {fig_path}")

# # Exemple d'appel de la fonction
# plot_mean_speed_with_sup_inf(all_metrics_df, path_save_pic)

In [ ]:
def plot_median_velocity_vs_time(all_metrics_df, path_save_pic, xlim: list = None):
    """
    Plot median velocities as a function of incubation time using all_metrics_df.
    Adds a trendline with a transparent error band to the data.

    Parameters:
    - all_metrics_df (DataFrame): DataFrame containing aggregated metrics for each experiment.
    - path_save_pic (str): Path where the plot will be saved.
    - xlim (list, optional): Limit for the x-axis.
    """
    # Trier les données par temps d'incubation
    all_metrics_sorted = all_metrics_df.sort_values(by='time to incubation (hours)')

    # Extraire les données
    x = all_metrics_sorted['time to incubation (hours)'].values
    y = all_metrics_sorted['mean_speed [um/min]'].values

    # Préparer le tracé
    plt.figure(figsize=(10, 5))
    plt.scatter(x, y, color='forestgreen', label='Médiane')  # Points
    plt.plot(x, y, linestyle='--', color='limegreen', alpha=0.6)  # Ligne connectant les points

    # Régression linéaire (y = ax + b)
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    trendline = slope * x + intercept

    # Calcul de la zone d'erreur
    error_band = 100*std_err * np.sqrt(1 / len(x) + (x - np.mean(x))**2 / np.sum((x - np.mean(x))**2))
    # Ajouter la ligne de tendance
    plt.plot(x, trendline, color='red', label=f"Tendance (y = {slope:.2f}x + {intercept:.2f})", linestyle='-', linewidth=1.5)

    # Ajouter l'aire d'erreur
    plt.fill_between(x, trendline - error_band, trendline + error_band, color='red', alpha=0.2, label="Marge d'erreur")

    # Ajouter des labels et des titres
    plt.title('Mean Speed vs. Time to Incubation', fontsize=14)
    plt.xlabel('Time to Incubation (hours)', fontsize=12)
    plt.ylabel('Mean Speed [μm/min]', fontsize=12)
    plt.ylim([0, 15])
    if xlim:
        plt.xlim(xlim)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(fontsize=10)

    # Sauvegarder et afficher
    plt.tight_layout()
    fig_path = os.path.join(path_save_pic, 'Mean_Speed_vs_Time_with_Trendline_and_ErrorBand.png')
    plt.savefig(fig_path, format='png')
    plt.show()

    print(f"Plot saved to {fig_path}")

#utilisation
plot_median_velocity_vs_time(all_metrics_df, path_save_pic)

In [ ]:
def plot_median_velocity_vs_time(all_metrics_df, path_save_pic, xlim: list = None):
    """
    Plot median velocities as a function of incubation time using all_metrics_df.

    Parameters:
    - all_metrics_df (DataFrame): DataFrame containing aggregated metrics for each experiment.
    - path_save_pic (str): Path where the plot will be saved.
    """
    # Trier les données par temps d'incubation
    all_metrics_sorted = all_metrics_df.sort_values(by='time to incubation (hours)')

    # Préparer le tracé
    plt.figure(figsize=(10, 5))
    plt.scatter(all_metrics_sorted['time to incubation (hours)'], 
                all_metrics_sorted['mean_speed [um/min]'],  # Correction de la colonne utilisée
                color='forestgreen', label='Médiane')
    plt.plot(all_metrics_sorted['time to incubation (hours)'], 
             all_metrics_sorted['mean_speed [um/min]'],  # Correction de la colonne utilisée
             linestyle='--', color='limegreen', alpha=0.6)

    # Ajouter des labels et des titres
    plt.title('Mean Speed vs. Time to Incubation', fontsize=14)
    plt.xlabel('Time to Incubation (hours)', fontsize=12)
    plt.ylim([0, 15])
    plt.ylabel('Mean Speed [μm/min]', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)

    # Sauvegarder et afficher
    plt.tight_layout()
    fig_path = os.path.join(path_save_pic, 'Mean_Speed_vs_Time.png')
    plt.savefig(fig_path, format='png')
    plt.show()

    print(f"Plot saved to {fig_path}")

# Appel de la fonction corrigée
plot_median_velocity_vs_time(all_metrics_df, path_save_pic)

In [ ]:
def plot_proportion_vs_time(all_metrics_df, path_save_pic, img_type):
    """
    Plot the proportion of low coefficient particles (proportion_inf) vs. time to incubation.

    Parameters:
    - all_metrics_df (DataFrame): DataFrame containing aggregated metrics for each experiment.
    - path_save_pic (str): Path where the plot will be saved.
    - img_type (str): Image format for saving the plot.
    """
    # Trier les données par temps d'incubation
    all_metrics_sorted = all_metrics_df.sort_values(by='time to incubation (hours)')

    # Préparer le tracé
    plt.figure(figsize=(10, 5))
    plt.scatter(all_metrics_sorted['time to incubation (hours)'], 
                all_metrics_sorted['proportion_inf'], 
                color='red', marker='+', label='Proportion inf')
    
    # Ajouter des titres et des labels
    plt.title('Proportion of Low Coefficient Particles vs. Time to Incubation', fontsize=14)
    plt.xlabel('Time to Incubation (hours)', fontsize=12)
    plt.ylabel('Proportion of Low Coefficient Particles', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)

    # Ajuster et sauvegarder
    plt.tight_layout()
    fig_path = os.path.join(path_save_pic, f'Proportion_Low_Coeff_Particles_vs_Time.{img_type}')
    plt.savefig(fig_path, format=img_type)
    plt.show()

    print(f"Plot saved to {fig_path}")

# Appel de la fonction
plot_proportion_vs_time(all_metrics_df, path_save_pic, IMG_TYPE)

In [ ]:
def plot_proportion_per_time_bins(all_metrics_df, path_save_pic, img_type):
    """
    Plot proportions of particles (Inf and Sup) per incubation time bins.

    Parameters:
    - all_metrics_df (DataFrame): DataFrame containing aggregated metrics for each experiment.
    - path_save_pic (str): Path where the plot will be saved.
    - img_type (str): Image format for saving the plot.
    """
    # Définir les intervalles de temps d'incubation
    bins = np.arange(0, all_metrics_df['time to incubation (hours)'].max() + 10, 10)
    labels = [f"{int(left)}-{int(right)}" for left, right in zip(bins[:-1], bins[1:])]
    all_metrics_df['time_bin'] = pd.cut(all_metrics_df['time to incubation (hours)'], bins=bins, labels=labels, right=False)

    # Calculer les proportions moyennes pour chaque bin
    proportion_per_bin_inf = all_metrics_df.groupby('time_bin')['proportion_inf'].mean()
    proportion_per_bin_sup = 1 - proportion_per_bin_inf

    # Configuration des barres pour le tracé
    x = np.arange(len(proportion_per_bin_inf))
    width = 0.35

    # Tracé des proportions
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.bar(x - width/2, proportion_per_bin_inf, width, color='red', alpha=0.7, label='Proportion Inf')
    ax.bar(x + width/2, proportion_per_bin_sup, width, color='blue', alpha=0.7, label='Proportion Sup')
    ax.set_title('Proportion of Particles per Incubation Time Bin', fontsize=14)
    ax.set_xlabel('Incubation Time Bin (hours)', fontsize=12)
    ax.set_ylabel('Average Proportion of Particles', fontsize=12)
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45)
    ax.legend()
    plt.grid(True, linestyle='--', alpha=0.6)

    # Sauvegarder et afficher le graphique
    plt.tight_layout()
    fig_path = os.path.join(path_save_pic, f'Proportion_vs_Time_Bins.{img_type}')
    plt.savefig(fig_path, format=img_type)
    plt.show()

    print(f"Plot saved to {fig_path}")

# Appel de la fonction
plot_proportion_per_time_bins(all_metrics_df, path_save_pic, IMG_TYPE)

In [ ]:
def plot_proportion_per_time_bins(all_metrics_df, DATA, path_save_pic, img_type):
    import os
    import numpy as np
    import matplotlib.pyplot as plt

    # Créer les bins
    bins = np.arange(0, all_metrics_df['time to incubation (hours)'].max() + 10, 10)
    labels = [f"{int(left)}-{int(right)}" for left, right in zip(bins[:-1], bins[1:])]
    
    # Appliquer les bins
    all_metrics_df['time_bin'] = pd.cut(all_metrics_df['time to incubation (hours)'], bins=bins, labels=labels, right=False)
    DATA['time_bin'] = pd.cut(DATA['time to incubation (hours)'], bins=bins, labels=labels, right=False)

    # Moyennes des proportions (métriques par expérience)
    proportion_per_bin_inf = all_metrics_df.groupby('time_bin')['proportion_inf'].mean().reindex(labels, fill_value=0)
    proportion_per_bin_sup = 1 - proportion_per_bin_inf

    # NOMBRE DE PARTICULES UNIQUES PAR BIN (total, low, high)
    total, n_low, n_high = compute_particle_counts_per_bin(DATA, bins, labels)

    # NOMBRE D'EXPÉRIENCES PAR BIN
    count_exp_per_bin = all_metrics_df.groupby('time_bin').size().reindex(labels, fill_value=0)

    # PLOT
    x = np.arange(len(labels))
    width = 0.3

    fig, ax1 = plt.subplots(figsize=(15, 7))
    bars_inf = ax1.bar(x - width/2, proportion_per_bin_inf, width, color='red', alpha=0.7, label='Proportion Inf (Low)')
    bars_sup = ax1.bar(x + width/2, proportion_per_bin_sup, width, color='blue', alpha=0.7, label='Proportion Sup (High)')
    ax1.set_ylabel('Average Proportion of Particles', fontsize=12)
    ax1.set_xlabel('Incubation Time Bin (hours)', fontsize=12)
    ax1.set_xticks(x)
    ax1.set_xticklabels(labels, rotation=45)
    ax1.set_ylim(0, 1)
    ax1.legend(loc='upper left')

    # Second axe : nombre de particules étudiées par bin
    ax2 = ax1.twinx()
    ax2.plot(x, total, 'ko-', label='N Particules (total)')
    ax2.plot(x, n_low, 'r.-', label='N Low')
    ax2.plot(x, n_high, 'b.-', label='N High')
    ax2.set_ylabel('Number of Unique Particles', fontsize=12)
    ax2.set_ylim(0, max(total.max(), n_low.max(), n_high.max()) * 1.2)
    ax2.legend(loc='upper right')

    # Annoter le nombre d’expériences sous chaque bin
    for i, count_exp in enumerate(count_exp_per_bin):
        ax1.annotate(f"Exp: {count_exp}", (x[i], 0), textcoords="offset points", xytext=(0, 10),
                     ha='center', fontsize=9, color='k', rotation=90)
    # Annoter n_low/n_high sous les barres
    for i in range(len(x)):
        txt = f"Low: {int(n_low.iloc[i])}\nHigh: {int(n_high.iloc[i])}"
        ax1.annotate(txt, (x[i], 0), textcoords="offset points", xytext=(0, -35),
                     ha='center', fontsize=9, color='dimgray', rotation=0)

    plt.title('Proportion of Particles & Counts per Incubation Time Bin', fontsize=15)
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.tight_layout()

    # Sauvegarder et afficher
    fig_path = os.path.join(path_save_pic, f'Proportion_vs_Time_Bins.{img_type}')
    plt.savefig(fig_path, format=img_type)
    plt.show()
    print(f"Plot saved to {fig_path}")

def compute_particle_counts_per_bin(DATA, bins, labels):
    # Attribuer la time_bin à chaque point
    DATA['time_bin'] = pd.cut(DATA['time to incubation (hours)'], bins=bins, labels=labels, right=False)

    # Grouper par bin ET particule, puis aggréger
    gb = DATA.groupby(['time_bin', 'particle']).agg(
        any_inf=('is_inf', 'any'),  # True si la particule a au moins un point "inf" dans ce bin
        any_sup=('is_sup', 'any')
    ).reset_index()

    # Nombre total de particules par bin
    total = gb.groupby('time_bin')['particle'].nunique().reindex(labels, fill_value=0)
    # Nombre "low"
    n_low = gb.groupby('time_bin')['any_inf'].sum().reindex(labels, fill_value=0)
    # Nombre "high"
    n_high = gb.groupby('time_bin')['any_sup'].sum().reindex(labels, fill_value=0)

    return total, n_low, n_high

# Appel :
plot_proportion_per_time_bins(all_metrics_df, DATA, path_save_pic, IMG_TYPE)

In [ ]:
def plot_proportion_per_time_bins(all_metrics_df, path_save_pic, img_type):
    """
    Plot proportions of particles (Inf and Sup) per incubation time bins, and show number of experiments per bin.
    """
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import os

    # Définir les intervalles de temps d'incubation
    bins = np.arange(0, all_metrics_df['time to incubation (hours)'].max() + 10, 10)
    labels = [f"{int(left)}-{int(right)}" for left, right in zip(bins[:-1], bins[1:])]
    all_metrics_df['time_bin'] = pd.cut(all_metrics_df['time to incubation (hours)'], bins=bins, labels=labels, right=False)

    # Calculer les proportions moyennes pour chaque bin
    proportion_per_bin_inf = all_metrics_df.groupby('time_bin')['proportion_inf'].mean()
    proportion_per_bin_sup = 1 - proportion_per_bin_inf

    # Calculer le nombre d’expériences par bin
    count_per_bin = all_metrics_df.groupby('time_bin').size()

    # Configuration des barres pour le tracé
    x = np.arange(len(proportion_per_bin_inf))
    width = 0.35

    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Barres proportions (axe principal)
    bars_inf = ax1.bar(x - width/2, proportion_per_bin_inf, width, color='red', alpha=0.7, label='Proportion Inf')
    bars_sup = ax1.bar(x + width/2, proportion_per_bin_sup, width, color='blue', alpha=0.7, label='Proportion Sup')
    ax1.set_ylabel('Average Proportion of Particles', fontsize=12, color='black')
    ax1.set_xlabel('Incubation Time Bin (hours)', fontsize=12)
    ax1.set_xticks(x)
    ax1.set_xticklabels(labels, rotation=45)
    ax1.set_ylim(0, 1)
    ax1.legend(loc='upper left')

    # Axe secondaire pour le nombre d’expériences
    ax2 = ax1.twinx()
    ax2.plot(x, count_per_bin, 'ko-', label='N Expériences')
    ax2.set_ylabel('Number of Experiments', fontsize=12, color='k')
    ax2.set_ylim(0, count_per_bin.max() * 1.2)
    ax2.legend(loc='upper right')

    # Annoter le nombre d’expériences sous chaque bin
    for i, count in enumerate(count_per_bin):
        ax2.annotate(str(count), (x[i], count), textcoords="offset points", xytext=(0,5),
                     ha='center', fontsize=10, color='black')

    plt.title('Proportion of Particles per Incubation Time Bin\nand Number of Experiments', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.tight_layout()

    # Sauvegarder et afficher le graphique
    fig_path = os.path.join(path_save_pic, f'Proportion_vs_Time_Bins.{img_type}')
    plt.savefig(fig_path, format=img_type)
    plt.show()

    print(f"Plot saved to {fig_path}")

# Appel de la fonction
plot_proportion_per_time_bins(all_metrics_df, path_save_pic, IMG_TYPE)

In [ ]:
# Plot displacement comparison between low and high coefficient particles
lib.plot_displacement_low_and_high(
    traj_sup=df_sup,
    traj_inf=df_inf,
    part_coef_inf=PART_COEF_INF,
    part_coef_sup=PART_COEF_SUP,
    start_end=start_end,
    save=True,
    pathway_saving=path_save_pic,
    name="displacement_start-end_time",
    img_type=IMG_TYPE
)

In [ ]:
def max_displacement_from_start(traj, size_pix):
    """
    Calcule la distance maximale entre le point de départ et toutes les positions
    atteintes par chaque particule.

    Parameters
    ----------
    traj : pandas.DataFrame
        Trajectoire des particules avec colonnes 'x', 'y', 'particle'.
    size_pix : float
        Taille d'un pixel en micromètres.

    Returns
    -------
    max_displacements : pandas.Series
        Distance maximale pour chaque particule.
    """
    # Identifier les coordonnées de départ pour chaque particule
    start_positions = traj.groupby('particle')[['x', 'y']].first()

    # Ajouter les coordonnées de départ au DataFrame
    traj = traj.join(start_positions, on='particle', rsuffix='_start')

    # Calculer les distances à partir du point de départ
    traj['distance_from_start [um]'] = size_pix * np.sqrt(
        (traj['x'] - traj['x_start'])**2 + 
        (traj['y'] - traj['y_start'])**2
    )

    # Trouver la distance maximale pour chaque particule
    max_displacements = traj.groupby('particle')['distance_from_start [um]'].max()

    return max_displacements

# Calcul des distances maximales
max_distances = max_displacement_from_start(DATA, size_pix=SIZE_PIX)

# Filtrage des particules appartenant à PART_COEF_INF et PART_COEF_SUP
distances_part_coef_inf = max_distances[max_distances.index.isin(PART_COEF_INF)]
distances_part_coef_sup = max_distances[max_distances.index.isin(PART_COEF_SUP)]

# Tracer les deux histogrammes sur le même graphique
plt.figure(figsize=(12, 7))
plt.hist(distances_part_coef_inf, bins=100, color='red', alpha=0.5, label='PART_COEF_INF')
plt.hist(distances_part_coef_sup, bins=100, color='blue', alpha=0.5, label='PART_COEF_SUP')
plt.title('Histogramme des distances maximales par catégorie', fontsize=16)
plt.xlabel('Distance maximale [μm]', fontsize=14)
plt.ylabel('Nombre de particules', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

X = DATA[['mass', 'size', 'ecc']].dropna()
X_scaled = StandardScaler().fit_transform(X)
kmeans = KMeans(n_clusters=2, random_state=0).fit(X_scaled)
DATA['cluster'] = -1
DATA.loc[X.index, 'cluster'] = kmeans.labels_

plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=DATA, x='size', y='mass',
    hue='cluster', palette=['green', 'orange'],
    alpha=0.1, s=0.001, marker='+', linewidth=1.0
)
plt.title("K-means clustering (2 groupes)")
plt.grid()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Données préparées comme avant
X = DATA[['mass', 'size', 'ecc']].dropna()
X_scaled = StandardScaler().fit_transform(X)
kmeans = KMeans(n_clusters=2, random_state=0).fit(X_scaled)
DATA['cluster'] = -1
DATA.loc[X.index, 'cluster'] = kmeans.labels_

# Création des deux sous-graphes côte à côte
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharex=True, sharey=True)

for i in range(2):
    sns.scatterplot(
        data=DATA[DATA['cluster'] == i],
        x='size', y='mass',
        ax=axes[i],
        color='green' if i == 0 else 'orange',
        alpha=1, s=10, marker='+', linewidth=1
    )
    axes[i].set_title(f"Groupe {i}")
    axes[i].grid(True)

fig.suptitle("Visualisation des 2 groupes K-means", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Séparation des clusters en deux DataFrames
DATA_cluster0 = DATA[DATA['cluster'] == 0].copy()
DATA_cluster1 = DATA[DATA['cluster'] == 1].copy()

In [ ]:
# Recompute trajectory clustering with new cutoff
# DATA_intermediaire = DATA.copy()
DATA_cluster0['frame'] = pd.factorize(DATA_cluster0['frame'])[0]
IMSD_cluster0 = tp.imsd(traj=DATA_cluster0, mpp=SIZE_PIX, fps=FPS, max_lagtime=200, statistic='msd')
LAG_TIME_FIT = 5
COEF_INF_cluster0, COEF_SUP_cluster0, PART_COEF_INF_cluster0, PART_COEF_SUP_cluster0, CUTOFF_cluster0 = lib.traj_clustering_with_fit_cutoff(
    DATA_cluster0, imsd=IMSD_cluster0, hist=True, lag_time_fit_start=0, lag_time_fit=LAG_TIME_FIT , micronperpixel=SIZE_PIX,
    fps=FPS, binsize=250, peak_height=50, peak_width=1, save=True, pathway_fig=path_save_pic,
    name='MSD and slopes cluster1', img_type=IMG_TYPE, plot=True, color_sup_inf=color_sup_inf,
    cutoff_default=0.6
)

# Ajouter les colonnes 'is_inf' et 'is_sup' à DATA
DATA_cluster0['particle'] = DATA_cluster0['particle'].astype(int)
PART_COEF_INF_cluster0 = set(map(int, PART_COEF_INF_cluster0))
PART_COEF_SUP_cluster0 = set(map(int, PART_COEF_SUP_cluster0))

# Ajout des colonnes 'is_inf' et 'is_sup'
DATA_cluster0['is_inf'] = DATA_cluster0['particle'].isin(PART_COEF_INF_cluster0).astype(int)
DATA_cluster0['is_sup'] = DATA_cluster0['particle'].isin(PART_COEF_SUP_cluster0).astype(int)


In [ ]:
# Recompute trajectory clustering with new cutoff
# DATA_intermediaire = DATA.copy()
DATA_cluster1['frame'] = pd.factorize(DATA_cluster1['frame'])[0]
IMSD_cluster1 = tp.imsd(traj=DATA_cluster1, mpp=SIZE_PIX, fps=FPS, max_lagtime=200, statistic='msd')
LAG_TIME_FIT = 5
COEF_INF_cluster1, COEF_SUP_cluster1, PART_COEF_INF_cluster1, PART_COEF_SUP_cluster1, CUTOFF_cluster1 = lib.traj_clustering_with_fit_cutoff(
    DATA_cluster1, imsd=IMSD_cluster1, hist=True, lag_time_fit_start=0, lag_time_fit=LAG_TIME_FIT , micronperpixel=SIZE_PIX,
    fps=FPS, binsize=250, peak_height=50, peak_width=1, save=True, pathway_fig=path_save_pic,
    name='MSD and slopes cluster2', img_type=IMG_TYPE, plot=True, color_sup_inf=color_sup_inf,
    cutoff_default=0.6
)
# Ajouter les colonnes 'is_inf' et 'is_sup' à DATA
DATA_cluster1['particle'] = DATA_cluster1['particle'].astype(int)
PART_COEF_INF_cluster1 = set(map(int, PART_COEF_INF_cluster1))
PART_COEF_SUP_cluster1 = set(map(int, PART_COEF_SUP_cluster1))
# Ajout des colonnes 'is_inf' et 'is_sup'
DATA_cluster1['is_inf'] = DATA_cluster1['particle'].isin(PART_COEF_INF_cluster1).astype(int)
DATA_cluster1['is_sup'] = DATA_cluster1['particle'].isin(PART_COEF_SUP_cluster1).astype(int)



In [ ]:
# for experiment in data_unique_pos['experiment'].unique():
#     # Sous-ensemble pour cette expérience
#     sub_df = data_unique_pos[data_unique_pos['experiment'] == experiment].copy()

#     # Filtrage spatial : éliminer particules trop proches
#     coords = sub_df[['x', 'y']].to_numpy()
#     if len(coords) > 1:
#         nbrs = NearestNeighbors(n_neighbors=2).fit(coords)
#         distances, _ = nbrs.kneighbors(coords)
#         keep_mask = distances[:, 1] > min_distance
#         sub_df = sub_df[keep_mask].copy()

#     # Recherche du dossier correspondant à l'expérience
#     experiment_dirs = glob.glob(os.path.join(base_image_dir, f"*{experiment}*"))
#     if not experiment_dirs:
#         print(f"Dossier pour {experiment} non trouvé !")
#         continue
#     experiment_dir = experiment_dirs[0]

#     # Chargement de l'image frame 0
#     img_path = os.path.join(experiment_dir, "mosaic/mosaic_total_0.tif")
#     if not os.path.isfile(img_path):
#         print(f"Image {img_path} non trouvée !")
#         continue

#     img = cv2.imread(img_path)
#     if img is None:
#         print(f"Image {img_path} illisible !")
#         continue

#     # Tracer les cercles
#     for _, row in sub_df.iterrows():
#         x = int(round(row["x"]))
#         y = int(round(row["y"]))
#         cv2.circle(img, (x, y), 50, (0, 0, 255), 3)

#     # Sauvegarde
#     out_path = os.path.join(output_dir, f"{experiment}_mosaic_total_0_inf_filtered.png")
#     cv2.imwrite(out_path, img)
#     print(f"Image annotée sauvegardée : {out_path}")

In [ ]:
# # === PARAMÈTRES ===
# base_image_dir = "/Users/souchaud/Desktop/A_analyser/CytoOne_HL5_10x_faits/"
# output_dir = "/Users/souchaud/Desktop/faibles/"
# min_distance = 10  # en pixels
# os.makedirs(output_dir, exist_ok=True)

# # === 1. Extraire particules du cluster 1, faibles ===
# data_inf = DATA[(DATA['cluster'] == 1) & (DATA['is_inf'] == 1)].copy()

# # === 2. Trouver la première frame de chaque particule ===
# first_frame_df = data_inf.sort_values('frame').groupby('particle', as_index=False).first()
# first_frame_df['start_frame'] = first_frame_df['frame']  # juste pour clarté

# # === 3. Filtrage spatial dans chaque frame de début ===
# filtered_particles = []

# for f in first_frame_df['start_frame'].unique():
#     same_frame = first_frame_df[first_frame_df['start_frame'] == f].copy()
    
#     if len(same_frame) <= 1:
#         filtered_particles.append(same_frame)
#         continue

#     coords = same_frame[['x', 'y']].to_numpy()
#     nbrs = NearestNeighbors(n_neighbors=2).fit(coords)
#     distances, _ = nbrs.kneighbors(coords)
#     keep_mask = distances[:, 1] > min_distance
#     filtered_particles.append(same_frame[keep_mask])

# # === 4. Particules uniques à annoter ===
# final_df = pd.concat(filtered_particles, ignore_index=True)

# # === 5. Génération des images par expérience ===
# for experiment in final_df['experiment'].unique():
#     sub_df = final_df[final_df['experiment'] == experiment].copy()

#     # Localise le dossier de l’expérience
#     experiment_dirs = glob.glob(os.path.join(base_image_dir, f"*{experiment}*"))
#     if not experiment_dirs:
#         print(f"Dossier pour {experiment} non trouvé !")
#         continue
#     experiment_dir = experiment_dirs[0]

#     # Charge l'image frame 0
#     img_path = os.path.join(experiment_dir, "mosaic/mosaic_total_0.tif")
#     if not os.path.isfile(img_path):
#         print(f"Image {img_path} non trouvée !")
#         continue

#     img = cv2.imread(img_path)
#     if img is None:
#         print(f"Image {img_path} illisible !")
#         continue

#     # Trace les cercles
#     for _, row in sub_df.iterrows():
#         x = int(round(row["x"]))
#         y = int(round(row["y"]))
#         cv2.circle(img, (x, y), 50, (0, 0, 255), 3)

#     # Sauvegarde
#     out_path = os.path.join(output_dir, f"{experiment}_mosaic_total_0_inf_filtered.png")
#     cv2.imwrite(out_path, img)
#     print(f"Image annotée sauvegardée : {out_path}")

In [ ]:
# On prend le cluster majoritaire par particule
df_particles = DATA.groupby('particle')['cluster'].agg(lambda x: x.value_counts().index[0]).reset_index()

# On compte le nombre de particules annotées 0 en cluster
n_particles_cluster0 = (df_particles['cluster'] == 1).sum()

print(f"Nombre de particules annotées cluster 0 : {n_particles_cluster0}")

In [ ]:
# Pour chaque particule, associer le cluster majoritaire, et la classe annotée (is_sup, is_inf)
df_comp = DATA.groupby('particle').agg({
    'cluster': lambda x: x.value_counts().index[0],  # cluster le plus fréquent
    'is_sup': 'first',    # ou 'any' si tu as de rares cellules mixtes
    'is_inf': 'first'
}).reset_index()

In [ ]:
# Exemple : correspondance cluster <-> is_sup
# (on suppose cluster 1 doit matcher is_sup=True, à vérifier selon ton contexte)
correspondance_sup = df_comp[df_comp['cluster'] == 0]['is_sup'].mean()
correspondance_inf = df_comp[df_comp['cluster'] == 1]['is_inf'].mean()

print(f"Taux de cellules du cluster 1 annotées SUP : {correspondance_sup:.2%}")
print(f"Taux de cellules du cluster 0 annotées INF : {correspondance_inf:.2%}")

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(DATA['size'], DATA['mass'], DATA['ecc'], c=DATA['cluster'], cmap='Set1', alpha=0.4, marker='.', s=1)
ax.set_xlabel('size')
ax.set_ylabel('mass')
ax.set_zlabel('ecc')
plt.show()

In [ ]:
def plot_speed_vs_density(all_metrics_df, save=False, pathway_saving=None, img_type="png"):
    """
    Trace la vitesse moyenne des particules en fonction de la densité cellulaire,
    en utilisant uniquement Matplotlib et des croix pour les points.

    Parameters
    ----------
    all_metrics_df : pandas.DataFrame
        Contient les colonnes 'mean_speed [um/min]' et 'nombre_part_par_champs'.
    save : bool, optional
        Si True, sauvegarde le graphique.
    pathway_saving : str, optional
        Chemin pour sauvegarder le graphique si save=True.
    img_type : str, optional
        Format d'image pour la sauvegarde (png, jpg, etc.).

    Returns
    -------
    None
    """
    # Vérification des colonnes nécessaires
    if not all(col in all_metrics_df.columns for col in ['mean_speed [um/min]', 'nombre_part_par_champs']):
        raise ValueError("Les colonnes 'mean_speed [um/min]' et 'nombre_part_par_champs' doivent être présentes dans all_metrics_df.")
    
    # Créer le graphique
    plt.figure(figsize=(10, 6))
    ax = plt.gca()  # Récupérer les axes actuels avec le style Matplotlib

    # Tracé des points avec des croix
    ax.scatter(
        all_metrics_df['nombre_part_par_champs'],
        all_metrics_df['mean_speed [um/min]'],
        s=100,  # Taille des symboles
        alpha=0.7,
        color='#1f77b4',  # Couleur compatible avec axes.prop_cycle
        marker='+',  # Utiliser des croix
        label="Points"
    )

    # Régression linéaire
    x = all_metrics_df['nombre_part_par_champs']
    y = all_metrics_df['mean_speed [um/min]']
    coeffs = np.polyfit(x, y, 1)  # Ajustement linéaire
    y_fit = np.polyval(coeffs, x)
    ax.plot(x, y_fit, color='#ff7f0e', label="Régression linéaire")  # Ligne de tendance

    # Titre et axes
    ax.set_title("Vitesse moyenne vs Densité cellulaire", fontsize=16, color='white')
    ax.set_xlabel("Nombre de particules par champ", fontsize=14, color='white')
    ax.set_ylabel("Vitesse moyenne [µm/min]", fontsize=14, color='white')
    ax.legend(fontsize=12)
    ax.grid(alpha=0.4)

    # Sauvegarder ou afficher
    if save and pathway_saving:
        filename = f"{pathway_saving}/mean_speed_vs_density.{img_type}"
        plt.savefig(filename, format=img_type, bbox_inches="tight", facecolor=plt.rcParams["figure.facecolor"])
        print(f"Graphique sauvegardé : {filename}")
    plt.show()

# Exemple d'utilisation
plot_speed_vs_density(
    all_metrics_df=all_metrics_df,
    save=False,
    pathway_saving=None,
    img_type="png"
)

In [ ]:
def plot_speed_vs_density(all_metrics_df, save=False, pathway_saving=None, img_type="png"):
    """
    Trace la vitesse moyenne des particules en fonction de la densité cellulaire,
    en respectant les paramètres Matplotlib globaux.

    Parameters
    ----------
    all_metrics_df : pandas.DataFrame
        Contient les colonnes 'mean_speed [um/min]' et 'nombre_part_par_champs'.
    save : bool, optional
        Si True, sauvegarde le graphique.
    pathway_saving : str, optional
        Chemin pour sauvegarder le graphique si save=True.
    img_type : str, optional
        Format d'image pour la sauvegarde (png, jpg, etc.).

    Returns
    -------
    None
    """
    # Vérification des colonnes nécessaires
    if not all(col in all_metrics_df.columns for col in ['mean_speed [um/min]', 'nombre_part_par_champs']):
        raise ValueError("Les colonnes 'mean_speed [um/min]' et 'nombre_part_par_champs' doivent être présentes dans all_metrics_df.")
    
    # Désactiver le style Seaborn pour respecter Matplotlib
    sns.set_theme(style=None)

    # Créer le graphique
    plt.figure(figsize=(10, 6))
    ax = plt.gca()  # Récupérer les axes actuels avec le style Matplotlib

    # Tracé des points avec scatterplot
    sns.scatterplot(
        x='nombre_part_par_champs',
        y='mean_speed [um/min]',
        data=all_metrics_df,
        s=100,  # Taille des points
        alpha=0.7,
        color='#1f77b4',  # Couleur compatible avec axes.prop_cycle
        edgecolor='white'
    )

    # Ajouter une ligne de tendance avec régression
    sns.regplot(
        x='nombre_part_par_champs',
        y='mean_speed [um/min]',
        data=all_metrics_df,
        scatter=False,
        color='#ff7f0e',  # Deuxième couleur du cycle
        line_kws={'label': "Régression linéaire"}
    )

    # Titre et axes
    ax.set_title("Vitesse moyenne vs Densité cellulaire", fontsize=16, color='white')
    ax.set_xlabel("Nombre de particules par champ", fontsize=14, color='white')
    ax.set_ylabel("Vitesse moyenne [µm/min]", fontsize=14, color='white')
    ax.legend(fontsize=12)
    ax.grid(alpha=0.4)

    # Sauvegarder ou afficher
    if save and pathway_saving:
        filename = f"{pathway_saving}/mean_speed_vs_density.{img_type}"
        plt.savefig(filename, format=img_type, bbox_inches="tight", facecolor=plt.rcParams["figure.facecolor"])
        print(f"Graphique sauvegardé : {filename}")
    plt.show()

# utilisation
plot_speed_vs_density(
    all_metrics_df=all_metrics_df,
    save=False,
    pathway_saving=None,
    img_type="png"
)

In [ ]:
def add_condition_and_save_hdf5(data, condition, name, save_path):
    """
    Ajoute une colonne 'condition' à DATA et sauvegarde le DataFrame au format HDF5.

    Parameters:
    - DATA (pd.DataFrame): DataFrame auquel ajouter la colonne.
    - CONDITION_simple (str): Valeur à ajouter dans la colonne 'condition'.
    - save_path (str): Chemin pour sauvegarder le fichier HDF5.
    -name (str): détail du fichier enregistré

    Returns:
    - None
    """
    # Ajouter la colonne 'condition'
    data['condition'] = condition

    # Vérifier si le DataFrame est non vide
    if data.empty:
        raise ValueError("Le DataFrame est vide, aucune donnée à sauvegarder.")

    # Sauvegarder au format HDF5
    data.to_hdf(save_path + condition + f"_{name}.hdf5", key='DATA', mode='w', format='table')
    print(f"DataFrame {name} sauvegardé avec la colonne 'condition' à l'emplacement : {save_path}")

In [ ]:
add_condition_and_save_hdf5(data=DATA, condition=CONDITION_simple, name="data", save_path="/Users/souchaud/Desktop/Analyses/tables_resultats/")
add_condition_and_save_hdf5(data=all_metrics_df, condition=CONDITION_simple, name="all_metrics", save_path="/Users/souchaud/Desktop/Analyses/tables_resultats/")
add_condition_and_save_hdf5(data=metrics_df, condition=CONDITION_simple, name="metrics", save_path="/Users/souchaud/Desktop/Analyses/tables_resultats/")

In [ ]:
len(PART_COEF_INF)/(len(PART_COEF_INF)+len(PART_COEF_SUP))

In [ ]:
len(PART_COEF_SUP)/(len(PART_COEF_INF)+len(PART_COEF_SUP))